
<a id='whatsnew-0240'></a>

# What’s new in 0.24.0 (January 25, 2019)

The 0.24.x series of releases will be the last to support Python 2. Future feature
releases will support Python 3 only. See [Plan for dropping Python 2.7](install.ipynb#install-dropping-27) for more
details.

{{ header }}

This is a major release from 0.23.4 and includes a number of API changes, new
features, enhancements, and performance improvements along with a large number
of bug fixes.

Highlights include:

- [Optional Integer NA Support](#whatsnew-0240-enhancements-intna)  
- [New APIs for accessing the array backing a Series or Index](#whatsnew-0240-values-api)  
- [A new top-level method for creating arrays](#whatsnew-0240-enhancements-array)  
- [Store Interval and Period data in a Series or DataFrame](#whatsnew-0240-enhancements-interval)  
- [Support for joining on two MultiIndexes](#whatsnew-0240-enhancements-join-with-two-multiindexes)  


Check the [API Changes](#whatsnew-0240-api-breaking) and [deprecations](#whatsnew-0240-deprecations) before updating.

These are the changes in pandas 0.24.0. See release for a full changelog
including other versions of pandas.

## Enhancements


<a id='whatsnew-0240-enhancements-intna'></a>

### Optional integer NA support

Pandas has gained the ability to hold integer dtypes with missing values. This long requested feature is enabled through the use of [extension types](development/extending.ipynb#extending-extension-types).

>**Note**
>
>IntegerArray is currently experimental. Its API or implementation may
change without warning.

We can construct a `Series` with the specified dtype. The dtype string `Int64` is a pandas `ExtensionDtype`. Specifying a list or array using the traditional missing value
marker of `np.nan` will infer to integer dtype. The display of the `Series` will also use the `NaN` to indicate missing values in string outputs. ([GH20700](https://github.com/pandas-dev/pandas/issues/20700), [GH20747](https://github.com/pandas-dev/pandas/issues/20747), [GH22441](https://github.com/pandas-dev/pandas/issues/22441), [GH21789](https://github.com/pandas-dev/pandas/issues/21789), [GH22346](https://github.com/pandas-dev/pandas/issues/22346))

Operations on these dtypes will propagate `NaN` as other pandas operations.

These dtypes can operate as part of a `DataFrame`.

These dtypes can be merged, reshaped, and casted.

Reduction and groupby operations such as `sum` work.

The Integer NA support currently uses the capitalized dtype version, e.g. `Int8` as compared to the traditional `int8`. This may be changed at a future date.

See [Nullable integer data type](user_guide/integer_na.ipynb#integer-na) for more.


<a id='whatsnew-0240-values-api'></a>

### Accessing the values in a Series or Index

`Series.array` and `Index.array` have been added for extracting the array backing a
`Series` or `Index`. ([GH19954](https://github.com/pandas-dev/pandas/issues/19954), [GH23623](https://github.com/pandas-dev/pandas/issues/23623))

Historically, this would have been done with `series.values`, but with
`.values` it was unclear whether the returned value would be the actual array,
some transformation of it, or one of pandas custom arrays (like
`Categorical`). For example, with `PeriodIndex`, `.values` generates
a new ndarray of period objects each time.

If you need an actual NumPy array, use `Series.to_numpy()` or `Index.to_numpy()`.

For Series and Indexes backed by normal NumPy arrays, `Series.array` will return a
new `arrays.PandasArray`, which is a thin (no-copy) wrapper around a
[`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray). `PandasArray` isn’t especially useful on its own,
but it does provide the same interface as any extension array defined in pandas or by
a third-party library.

We haven’t removed or deprecated `Series.values` or `DataFrame.values`, but we
highly recommend and using `.array` or `.to_numpy()` instead.

See [Dtypes](getting_started/basics.ipynb#basics-dtypes) and [Attributes and Underlying Data](getting_started/basics.ipynb#basics-attrs) for more.


<a id='whatsnew-0240-enhancements-array'></a>

### `pandas.array`: a new top-level method for creating arrays

A new top-level method `array()` has been added for creating 1-dimensional arrays ([GH22860](https://github.com/pandas-dev/pandas/issues/22860)).
This can be used to create any [extension array](development/extending.ipynb#extending-extension-types), including
extension arrays registered by [3rd party libraries](ecosystem.ipynb#ecosystem-extensions).
See the [dtypes docs](getting_started/basics.ipynb#basics-dtypes) for more on extension arrays.

Passing data for which there isn’t dedicated extension type (e.g. float, integer, etc.)
will return a new `arrays.PandasArray`, which is just a thin (no-copy)
wrapper around a [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) that satisfies the pandas extension array interface.

On their own, a `PandasArray` isn’t a very useful object.
But if you need write low-level code that works generically for any
`ExtensionArray`, `PandasArray`
satisfies that need.

Notice that by default, if no `dtype` is specified, the dtype of the returned
array is inferred from the data. In particular, note that the first example of
`[1, 2, np.nan]` would have returned a floating-point array, since `NaN`
is a float.


<a id='whatsnew-0240-enhancements-interval'></a>

### Storing Interval and Period data in Series and DataFrame

`Interval` and `Period` data may now be stored in a `Series` or `DataFrame`, in addition to an
`IntervalIndex` and `PeriodIndex` like previously ([GH19453](https://github.com/pandas-dev/pandas/issues/19453), [GH22862](https://github.com/pandas-dev/pandas/issues/22862)).

For periods:

Previously, these would be cast to a NumPy array with object dtype. In general,
this should result in better performance when storing an array of intervals or periods
in a `Series` or column of a `DataFrame`.

Use `Series.array` to extract the underlying array of intervals or periods
from the `Series`:

These return an instance of `arrays.IntervalArray` or `arrays.PeriodArray`,
the new extension arrays that back interval and period data.

For backwards compatibility, `Series.values` continues to return
a NumPy array of objects for Interval and Period data. We recommend
using `Series.array` when you need the array of data stored in the
`Series`, and `Series.to_numpy()` when you know you need a NumPy array.

See [Dtypes](getting_started/basics.ipynb#basics-dtypes) and [Attributes and Underlying Data](getting_started/basics.ipynb#basics-attrs)
for more.


<a id='whatsnew-0240-enhancements-join-with-two-multiindexes'></a>

### Joining with two multi-indexes

`DataFrame.merge()` and `DataFrame.join()` can now be used to join multi-indexed `Dataframe` instances on the overlapping index levels ([GH6360](https://github.com/pandas-dev/pandas/issues/6360))

See the [Merge, join, and concatenate](user_guide/merging.ipynb#merging-join-with-two-multi-indexes) documentation section.

For earlier versions this can be done using the following.


<a id='whatsnew-0240-enhancements-read-html'></a>

### `read_html` Enhancements

`read_html()` previously ignored `colspan` and `rowspan` attributes.
Now it understands them, treating them as sequences of cells with the same
value. ([GH17054](https://github.com/pandas-dev/pandas/issues/17054))

*Previous behavior*:

```ipython
In [13]: result
Out [13]:
[   A  B   C
 0  1  2 NaN]
```


*New behavior*:

### New `Styler.pipe()` method

The `Styler` class has gained a
`pipe()` method.  This provides a
convenient way to apply users’ predefined styling functions, and can help reduce
“boilerplate” when using DataFrame styling functionality repeatedly within a notebook. ([GH23229](https://github.com/pandas-dev/pandas/issues/23229))

Similar methods already exist for other classes in pandas, including `DataFrame.pipe()`,
`GroupBy.pipe()`, and `Resampler.pipe()`.


<a id='whatsnew-0240-enhancements-rename-axis'></a>

### Renaming names in a MultiIndex

`DataFrame.rename_axis()` now supports `index` and `columns` arguments
and `Series.rename_axis()` supports `index` argument ([GH19978](https://github.com/pandas-dev/pandas/issues/19978)).

This change allows a dictionary to be passed so that some of the names
of a `MultiIndex` can be changed.

Example:

See the [Advanced documentation on renaming](user_guide/advanced.ipynb#advanced-index-names) for more details.


<a id='whatsnew-0240-enhancements-other'></a>

### Other enhancements

- `merge()` now directly allows merge between objects of type `DataFrame` and named `Series`, without the need to convert the `Series` object into a `DataFrame` beforehand ([GH21220](https://github.com/pandas-dev/pandas/issues/21220))  
- `ExcelWriter` now accepts `mode` as a keyword argument, enabling append to existing workbooks when using the `openpyxl` engine ([GH3441](https://github.com/pandas-dev/pandas/issues/3441))  
- `FrozenList` has gained the `.union()` and `.difference()` methods. This functionality greatly simplifies groupby’s that rely on explicitly excluding certain columns. See [Splitting an object into groups](user_guide/groupby.ipynb#groupby-split) for more information ([GH15475](https://github.com/pandas-dev/pandas/issues/15475), [GH15506](https://github.com/pandas-dev/pandas/issues/15506)).  
- `DataFrame.to_parquet()` now accepts `index` as an argument, allowing
  the user to override the engine’s default behavior to include or omit the
  dataframe’s indexes from the resulting Parquet file. ([GH20768](https://github.com/pandas-dev/pandas/issues/20768))  
- `read_feather()` now accepts `columns` as an argument, allowing the user to specify which columns should be read. ([GH24025](https://github.com/pandas-dev/pandas/issues/24025))  
- `DataFrame.corr()` and `Series.corr()` now accept a callable for generic calculation methods of correlation, e.g. histogram intersection ([GH22684](https://github.com/pandas-dev/pandas/issues/22684))  
- `DataFrame.to_string()` now accepts `decimal` as an argument, allowing the user to specify which decimal separator should be used in the output. ([GH23614](https://github.com/pandas-dev/pandas/issues/23614))  
- `DataFrame.to_html()` now accepts `render_links` as an argument, allowing the user to generate HTML with links to any URLs that appear in the DataFrame.
  See the [section on writing HTML](user_guide/io.ipynb#io-html) in the IO docs for example usage. ([GH2679](https://github.com/pandas-dev/pandas/issues/2679))  
- `pandas.read_csv()` now supports pandas extension types as an argument to `dtype`, allowing the user to use pandas extension types when reading CSVs. ([GH23228](https://github.com/pandas-dev/pandas/issues/23228))  
- The `shift()` method now accepts fill_value as an argument, allowing the user to specify a value which will be used instead of NA/NaT in the empty periods. ([GH15486](https://github.com/pandas-dev/pandas/issues/15486))  
- `to_datetime()` now supports the `%Z` and `%z` directive when passed into `format` ([GH13486](https://github.com/pandas-dev/pandas/issues/13486))  
- `Series.mode()` and `DataFrame.mode()` now support the `dropna` parameter which can be used to specify whether `NaN`/`NaT` values should be considered ([GH17534](https://github.com/pandas-dev/pandas/issues/17534))  
- `DataFrame.to_csv()` and `Series.to_csv()` now support the `compression` keyword when a file handle is passed. ([GH21227](https://github.com/pandas-dev/pandas/issues/21227))  
- `Index.droplevel()` is now implemented also for flat indexes, for compatibility with `MultiIndex` ([GH21115](https://github.com/pandas-dev/pandas/issues/21115))  
- `Series.droplevel()` and `DataFrame.droplevel()` are now implemented ([GH20342](https://github.com/pandas-dev/pandas/issues/20342))  
- Added support for reading from/writing to Google Cloud Storage via the `gcsfs` library ([GH19454](https://github.com/pandas-dev/pandas/issues/19454), [GH23094](https://github.com/pandas-dev/pandas/issues/23094))  
- `DataFrame.to_gbq()` and `read_gbq()` signature and documentation updated to
  reflect changes from the [Pandas-GBQ library version 0.8.0](https://pandas-gbq.readthedocs.io/en/latest/changelog.html#changelog-0-8-0).
  Adds a `credentials` argument, which enables the use of any kind of
  [google-auth credentials](https://google-auth.readthedocs.io/en/latest/). ([GH21627](https://github.com/pandas-dev/pandas/issues/21627),
  [GH22557](https://github.com/pandas-dev/pandas/issues/22557), [GH23662](https://github.com/pandas-dev/pandas/issues/23662))  
- New method `HDFStore.walk()` will recursively walk the group hierarchy of an HDF5 file ([GH10932](https://github.com/pandas-dev/pandas/issues/10932))  
- `read_html()` copies cell data across `colspan` and `rowspan`, and it treats all-`th` table rows as headers if `header` kwarg is not given and there is no `thead` ([GH17054](https://github.com/pandas-dev/pandas/issues/17054))  
- `Series.nlargest()`, `Series.nsmallest()`, `DataFrame.nlargest()`, and `DataFrame.nsmallest()` now accept the value `"all"` for the `keep` argument. This keeps all ties for the nth largest/smallest value ([GH16818](https://github.com/pandas-dev/pandas/issues/16818))  
- `IntervalIndex` has gained the `set_closed()` method to change the existing `closed` value ([GH21670](https://github.com/pandas-dev/pandas/issues/21670))  
- `to_csv()`, `to_csv()`, `to_json()`, and `to_json()` now support `compression='infer'` to infer compression based on filename extension ([GH15008](https://github.com/pandas-dev/pandas/issues/15008)).
  The default compression for `to_csv`, `to_json`, and `to_pickle` methods has been updated to `'infer'` ([GH22004](https://github.com/pandas-dev/pandas/issues/22004)).  
- `DataFrame.to_sql()` now supports writing `TIMESTAMP WITH TIME ZONE` types for supported databases. For databases that don’t support timezones, datetime data will be stored as timezone unaware local timestamps. See the [Datetime data types](user_guide/io.ipynb#io-sql-datetime-data) for implications ([GH9086](https://github.com/pandas-dev/pandas/issues/9086)).  
- `to_timedelta()` now supports iso-formated timedelta strings ([GH21877](https://github.com/pandas-dev/pandas/issues/21877))  
- `Series` and `DataFrame` now support `Iterable` objects in the constructor ([GH2193](https://github.com/pandas-dev/pandas/issues/2193))  
- `DatetimeIndex` has gained the `DatetimeIndex.timetz` attribute. This returns the local time with timezone information. ([GH21358](https://github.com/pandas-dev/pandas/issues/21358))  
- `round()`, `ceil()`, and `floor()` for `DatetimeIndex` and `Timestamp`
  now support an `ambiguous` argument for handling datetimes that are rounded to ambiguous times ([GH18946](https://github.com/pandas-dev/pandas/issues/18946))
  and a `nonexistent` argument for handling datetimes that are rounded to nonexistent times. See [Nonexistent times when localizing](user_guide/timeseries.ipynb#timeseries-timezone-nonexistent) ([GH22647](https://github.com/pandas-dev/pandas/issues/22647))  
- The result of `resample()` is now iterable similar to `groupby()` ([GH15314](https://github.com/pandas-dev/pandas/issues/15314)).  
- `Series.resample()` and `DataFrame.resample()` have gained the `pandas.core.resample.Resampler.quantile()` ([GH15023](https://github.com/pandas-dev/pandas/issues/15023)).  
- `DataFrame.resample()` and `Series.resample()` with a `PeriodIndex` will now respect the `base` argument in the same fashion as with a `DatetimeIndex`. ([GH23882](https://github.com/pandas-dev/pandas/issues/23882))  
- `pandas.api.types.is_list_like()` has gained a keyword `allow_sets` which is `True` by default; if `False`,
  all instances of `set` will not be considered “list-like” anymore ([GH23061](https://github.com/pandas-dev/pandas/issues/23061))  
- `Index.to_frame()` now supports overriding column name(s) ([GH22580](https://github.com/pandas-dev/pandas/issues/22580)).  
- `Categorical.from_codes()` now can take a `dtype` parameter as an alternative to passing `categories` and `ordered` ([GH24398](https://github.com/pandas-dev/pandas/issues/24398)).  
- New attribute `__git_version__` will return git commit sha of current build ([GH21295](https://github.com/pandas-dev/pandas/issues/21295)).  
- Compatibility with Matplotlib 3.0 ([GH22790](https://github.com/pandas-dev/pandas/issues/22790)).  
- Added `Interval.overlaps()`, `arrays.IntervalArray.overlaps()`, and `IntervalIndex.overlaps()` for determining overlaps between interval-like objects ([GH21998](https://github.com/pandas-dev/pandas/issues/21998))  
- `read_fwf()` now accepts keyword `infer_nrows` ([GH15138](https://github.com/pandas-dev/pandas/issues/15138)).  
- `to_parquet()` now supports writing a `DataFrame` as a directory of parquet files partitioned by a subset of the columns when `engine = 'pyarrow'` ([GH23283](https://github.com/pandas-dev/pandas/issues/23283))  
- `Timestamp.tz_localize()`, `DatetimeIndex.tz_localize()`, and `Series.tz_localize()` have gained the `nonexistent` argument for alternative handling of nonexistent times. See [Nonexistent times when localizing](user_guide/timeseries.ipynb#timeseries-timezone-nonexistent) ([GH8917](https://github.com/pandas-dev/pandas/issues/8917), [GH24466](https://github.com/pandas-dev/pandas/issues/24466))  
- `Index.difference()`, `Index.intersection()`, `Index.union()`, and `Index.symmetric_difference()` now have an optional `sort` parameter to control whether the results should be sorted if possible ([GH17839](https://github.com/pandas-dev/pandas/issues/17839), [GH24471](https://github.com/pandas-dev/pandas/issues/24471))  
- `read_excel()` now accepts `usecols` as a list of column names or callable ([GH18273](https://github.com/pandas-dev/pandas/issues/18273))  
- `MultiIndex.to_flat_index()` has been added to flatten multiple levels into a single-level `Index` object.  
- `DataFrame.to_stata()` and `pandas.io.stata.StataWriter117` can write mixed sting columns to Stata strl format ([GH23633](https://github.com/pandas-dev/pandas/issues/23633))  
- `DataFrame.between_time()` and `DataFrame.at_time()` have gained the `axis` parameter ([GH8839](https://github.com/pandas-dev/pandas/issues/8839))  
- `DataFrame.to_records()` now accepts `index_dtypes` and `column_dtypes` parameters to allow different data types in stored column and index records ([GH18146](https://github.com/pandas-dev/pandas/issues/18146))  
- `IntervalIndex` has gained the `is_overlapping` attribute to indicate if the `IntervalIndex` contains any overlapping intervals ([GH23309](https://github.com/pandas-dev/pandas/issues/23309))  
- [`pandas.DataFrame.to_sql()`](reference/api/pandas.DataFrame.to_sql.ipynb#pandas.DataFrame.to_sql) has gained the `method` argument to control SQL insertion clause. See the [insertion method](user_guide/io.ipynb#io-sql-method) section in the documentation. ([GH8953](https://github.com/pandas-dev/pandas/issues/8953))  
- `DataFrame.corrwith()` now supports Spearman’s rank correlation, Kendall’s tau as well as callable correlation methods. ([GH21925](https://github.com/pandas-dev/pandas/issues/21925))  
- `DataFrame.to_json()`, `DataFrame.to_csv()`, `DataFrame.to_pickle()`, and other export methods now support tilde(~) in path argument. ([GH23473](https://github.com/pandas-dev/pandas/issues/23473))  



<a id='whatsnew-0240-api-breaking'></a>

## Backwards incompatible API changes

Pandas 0.24.0 includes a number of API breaking changes.


<a id='whatsnew-0240-api-breaking-deps'></a>

### Increased minimum versions for dependencies

We have updated our minimum supported versions of dependencies ([GH21242](https://github.com/pandas-dev/pandas/issues/21242), [GH18742](https://github.com/pandas-dev/pandas/issues/18742), [GH23774](https://github.com/pandas-dev/pandas/issues/23774), [GH24767](https://github.com/pandas-dev/pandas/issues/24767)).
If installed, we now require:

|Package|Minimum Version|Required|
|:---------------:|:---------------:|:--------:|
|numpy|1.12.0|X|
|bottleneck|1.2.0||
|fastparquet|0.2.1||
|matplotlib|2.0.0||
|numexpr|2.6.1||
|pandas-gbq|0.8.0||
|pyarrow|0.9.0||
|pytables|3.4.2||
|scipy|0.18.1||
|xlrd|1.0.0||
|pytest (dev)|3.6||
Additionally we no longer depend on `feather-format` for feather based storage
and replaced it with references to `pyarrow` ([GH21639](https://github.com/pandas-dev/pandas/issues/21639) and [GH23053](https://github.com/pandas-dev/pandas/issues/23053)).


<a id='whatsnew-0240-api-breaking-csv-line-terminator'></a>

### os.linesep is used for `line_terminator` of `DataFrame.to_csv`

`DataFrame.to_csv()` now uses `os.linesep()` rather than `'\n'`
for the default line terminator ([GH20353](https://github.com/pandas-dev/pandas/issues/20353)).
This change only affects when running on Windows, where `'\r\n'` was used for line terminator
even when `'\n'` was passed in `line_terminator`.

*Previous behavior* on Windows:

```ipython
In [1]: data = pd.DataFrame({"string_with_lf": ["a\nbc"],
   ...:                      "string_with_crlf": ["a\r\nbc"]})

In [2]: # When passing file PATH to to_csv,
   ...: # line_terminator does not work, and csv is saved with '\r\n'.
   ...: # Also, this converts all '\n's in the data to '\r\n'.
   ...: data.to_csv("test.csv", index=False, line_terminator='\n')

In [3]: with open("test.csv", mode='rb') as f:
   ...:     print(f.read())
Out[3]: b'string_with_lf,string_with_crlf\r\n"a\r\nbc","a\r\r\nbc"\r\n'

In [4]: # When passing file OBJECT with newline option to
   ...: # to_csv, line_terminator works.
   ...: with open("test2.csv", mode='w', newline='\n') as f:
   ...:     data.to_csv(f, index=False, line_terminator='\n')

In [5]: with open("test2.csv", mode='rb') as f:
   ...:     print(f.read())
Out[5]: b'string_with_lf,string_with_crlf\n"a\nbc","a\r\nbc"\n'
```


*New behavior* on Windows:

Passing `line_terminator` explicitly, set thes `line terminator` to that character.

```ipython
In [1]: data = pd.DataFrame({"string_with_lf": ["a\nbc"],
   ...:                      "string_with_crlf": ["a\r\nbc"]})

In [2]: data.to_csv("test.csv", index=False, line_terminator='\n')

In [3]: with open("test.csv", mode='rb') as f:
   ...:     print(f.read())
Out[3]: b'string_with_lf,string_with_crlf\n"a\nbc","a\r\nbc"\n'
```


On Windows, the value of `os.linesep` is `'\r\n'`, so if `line_terminator` is not
set, `'\r\n'` is used for line terminator.

```ipython
In [1]: data = pd.DataFrame({"string_with_lf": ["a\nbc"],
   ...:                      "string_with_crlf": ["a\r\nbc"]})

In [2]: data.to_csv("test.csv", index=False)

In [3]: with open("test.csv", mode='rb') as f:
   ...:     print(f.read())
Out[3]: b'string_with_lf,string_with_crlf\r\n"a\nbc","a\r\nbc"\r\n'
```


For file objects, specifying `newline` is not sufficient to set the line terminator.
You must pass in the `line_terminator` explicitly, even in this case.

```ipython
In [1]: data = pd.DataFrame({"string_with_lf": ["a\nbc"],
   ...:                      "string_with_crlf": ["a\r\nbc"]})

In [2]: with open("test2.csv", mode='w', newline='\n') as f:
   ...:     data.to_csv(f, index=False)

In [3]: with open("test2.csv", mode='rb') as f:
   ...:     print(f.read())
Out[3]: b'string_with_lf,string_with_crlf\r\n"a\nbc","a\r\nbc"\r\n'
```



<a id='whatsnew-0240-bug-fixes-nan-with-str-dtype'></a>

### Proper handling of np.NaN in a string data-typed column with the Python engine

There was bug in `read_excel()` and `read_csv()` with the Python
engine, where missing values turned to `'nan'` with `dtype=str` and
`na_filter=True`. Now, these missing values are converted to the string
missing indicator, `np.nan`. ([GH20377](https://github.com/pandas-dev/pandas/issues/20377))

*Previous behavior*:

```ipython
In [5]: data = 'a,b,c\n1,,3\n4,5,6'
In [6]: df = pd.read_csv(StringIO(data), engine='python', dtype=str, na_filter=True)
In [7]: df.loc[0, 'b']
Out[7]:
'nan'
```


*New behavior*:

Notice how we now instead output `np.nan` itself instead of a stringified form of it.


<a id='whatsnew-0240-api-timezone-offset-parsing'></a>

### Parsing datetime strings with timezone offsets

Previously, parsing datetime strings with UTC offsets with `to_datetime()`
or `DatetimeIndex` would automatically convert the datetime to UTC
without timezone localization. This is inconsistent from parsing the same
datetime string with `Timestamp` which would preserve the UTC
offset in the `tz` attribute. Now, `to_datetime()` preserves the UTC
offset in the `tz` attribute when all the datetime strings have the same
UTC offset ([GH17697](https://github.com/pandas-dev/pandas/issues/17697), [GH11736](https://github.com/pandas-dev/pandas/issues/11736), [GH22457](https://github.com/pandas-dev/pandas/issues/22457))

*Previous behavior*:

```ipython
In [2]: pd.to_datetime("2015-11-18 15:30:00+05:30")
Out[2]: Timestamp('2015-11-18 10:00:00')

In [3]: pd.Timestamp("2015-11-18 15:30:00+05:30")
Out[3]: Timestamp('2015-11-18 15:30:00+0530', tz='pytz.FixedOffset(330)')

# Different UTC offsets would automatically convert the datetimes to UTC (without a UTC timezone)
In [4]: pd.to_datetime(["2015-11-18 15:30:00+05:30", "2015-11-18 16:30:00+06:30"])
Out[4]: DatetimeIndex(['2015-11-18 10:00:00', '2015-11-18 10:00:00'], dtype='datetime64[ns]', freq=None)
```


*New behavior*:

Parsing datetime strings with the same UTC offset will preserve the UTC offset in the `tz`

Parsing datetime strings with different UTC offsets will now create an Index of
`datetime.datetime` objects with different UTC offsets

Passing `utc=True` will mimic the previous behavior but will correctly indicate
that the dates have been converted to UTC


<a id='whatsnew-0240-api-breaking-read-csv-mixed-tz'></a>

### Parsing mixed-timezones with `read_csv()`

`read_csv()` no longer silently converts mixed-timezone columns to UTC ([GH24987](https://github.com/pandas-dev/pandas/issues/24987)).

*Previous behavior*

In [ ]:
>>> import io
>>> content = """\
... a
... 2000-01-01T00:00:00+05:00
... 2000-01-01T00:00:00+06:00"""
>>> df = pd.read_csv(io.StringIO(content), parse_dates=['a'])
>>> df.a
0   1999-12-31 19:00:00
1   1999-12-31 18:00:00
Name: a, dtype: datetime64[ns]

*New behavior*

As can be seen, the `dtype` is object; each value in the column is a string.
To convert the strings to an array of datetimes, the `date_parser` argument

See [Parsing datetime strings with timezone offsets](#whatsnew-0240-api-timezone-offset-parsing) for more.


<a id='whatsnew-0240-api-breaking-period-end-time'></a>

### Time values in `dt.end_time` and `to_timestamp(how='end')`

The time values in `Period` and `PeriodIndex` objects are now set
to ‘23:59:59.999999999’ when calling `Series.dt.end_time`, `Period.end_time`,
`PeriodIndex.end_time`, `Period.to_timestamp()` with `how='end'`,
or `PeriodIndex.to_timestamp()` with `how='end'` ([GH17157](https://github.com/pandas-dev/pandas/issues/17157))

*Previous behavior*:

```ipython
In [2]: p = pd.Period('2017-01-01', 'D')
In [3]: pi = pd.PeriodIndex([p])

In [4]: pd.Series(pi).dt.end_time[0]
Out[4]: Timestamp(2017-01-01 00:00:00)

In [5]: p.end_time
Out[5]: Timestamp(2017-01-01 23:59:59.999999999)
```


*New behavior*:

Calling `Series.dt.end_time` will now result in a time of ‘23:59:59.999999999’ as
is the case with `Period.end_time`, for example


<a id='whatsnew-0240-api-breaking-datetime-unique'></a>

### Series.unique for Timezone-Aware Data

The return type of `Series.unique()` for datetime with timezone values has changed
from an [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) of `Timestamp` objects to a `arrays.DatetimeArray` ([GH24024](https://github.com/pandas-dev/pandas/issues/24024)).

*Previous behavior*:

```ipython
In [3]: ser.unique()
Out[3]: array([Timestamp('2000-01-01 00:00:00+0000', tz='UTC')], dtype=object)
```


*New behavior*:


<a id='whatsnew-0240-api-breaking-sparse-values'></a>

### Sparse data structure refactor

`SparseArray`, the array backing `SparseSeries` and the columns in a `SparseDataFrame`,
is now an extension array ([GH21978](https://github.com/pandas-dev/pandas/issues/21978), [GH19056](https://github.com/pandas-dev/pandas/issues/19056), [GH22835](https://github.com/pandas-dev/pandas/issues/22835)).
To conform to this interface and for consistency with the rest of pandas, some API breaking
changes were made:

- `SparseArray` is no longer a subclass of [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray). To convert a `SparseArray` to a NumPy array, use [`numpy.asarray()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.asarray.html#numpy.asarray).  
- `SparseArray.dtype` and `SparseSeries.dtype` are now instances of `SparseDtype`, rather than `np.dtype`. Access the underlying dtype with `SparseDtype.subtype`.  
- `numpy.asarray(sparse_array)` now returns a dense array with all the values, not just the non-fill-value values ([GH14167](https://github.com/pandas-dev/pandas/issues/14167))  
- `SparseArray.take` now matches the API of `pandas.api.extensions.ExtensionArray.take()` ([GH19506](https://github.com/pandas-dev/pandas/issues/19506)):  
  - The default value of `allow_fill` has changed from `False` to `True`.  
  - The `out` and `mode` parameters are now longer accepted (previously, this raised if they were specified).  
  - Passing a scalar for `indices` is no longer allowed.  
- The result of `concat()` with a mix of sparse and dense Series is a Series with sparse values, rather than a `SparseSeries`.  
- `SparseDataFrame.combine` and `DataFrame.combine_first` no longer supports combining a sparse column with a dense column while preserving the sparse subtype. The result will be an object-dtype SparseArray.  
- Setting `SparseArray.fill_value` to a fill value with a different dtype is now allowed.  
- `DataFrame[column]` is now a `Series` with sparse values, rather than a `SparseSeries`, when slicing a single column with sparse values ([GH23559](https://github.com/pandas-dev/pandas/issues/23559)).  
- The result of `Series.where()` is now a `Series` with sparse values, like with other extension arrays ([GH24077](https://github.com/pandas-dev/pandas/issues/24077))  


Some new warnings are issued for operations that require or are likely to materialize a large dense array:

- A `errors.PerformanceWarning` is issued when using fillna with a `method`, as a dense array is constructed to create the filled array. Filling with a `value` is the efficient way to fill a sparse array.  
- A `errors.PerformanceWarning` is now issued when concatenating sparse Series with differing fill values. The fill value from the first sparse array continues to be used.  


In addition to these API breaking changes, many [Performance Improvements and Bug Fixes have been made](#whatsnew-0240-bug-fixes-sparse).

Finally, a `Series.sparse` accessor was added to provide sparse-specific methods like `Series.sparse.from_coo()`.


<a id='whatsnew-0240-api-breaking-get-dummies'></a>

### `get_dummies()` always returns a DataFrame

Previously, when `sparse=True` was passed to `get_dummies()`, the return value could be either
a `DataFrame` or a `SparseDataFrame`, depending on whether all or a just a subset
of the columns were dummy-encoded. Now, a `DataFrame` is always returned ([GH24284](https://github.com/pandas-dev/pandas/issues/24284)).

*Previous behavior*

The first `get_dummies()` returns a `DataFrame` because the column `A`
is not dummy encoded. When just `["B", "C"]` are passed to `get_dummies`,
then all the columns are dummy-encoded, and a `SparseDataFrame` was returned.

```ipython
In [2]: df = pd.DataFrame({"A": [1, 2], "B": ['a', 'b'], "C": ['a', 'a']})

In [3]: type(pd.get_dummies(df, sparse=True))
Out[3]: pandas.core.frame.DataFrame

In [4]: type(pd.get_dummies(df[['B', 'C']], sparse=True))
Out[4]: pandas.core.sparse.frame.SparseDataFrame
```


*New behavior*

Now, the return type is consistently a `DataFrame`.

>**Note**
>
>There’s no difference in memory usage between a `SparseDataFrame`
and a `DataFrame` with sparse values. The memory usage will
be the same as in the previous version of pandas.


<a id='whatsnew-0240-api-breaking-frame-to-dict-index-orient'></a>

### Raise ValueError in `DataFrame.to_dict(orient='index')`

Bug in `DataFrame.to_dict()` raises `ValueError` when used with
`orient='index'` and a non-unique index instead of losing data ([GH22801](https://github.com/pandas-dev/pandas/issues/22801))


<a id='whatsnew-0240-api-datetimelike-normalize'></a>

### Tick DateOffset normalize restrictions

Creating a `Tick` object (`Day`, `Hour`, `Minute`,
`Second`, `Milli`, `Micro`, `Nano`) with
`normalize=True` is no longer supported.  This prevents unexpected behavior
where addition could fail to be monotone or associative.  ([GH21427](https://github.com/pandas-dev/pandas/issues/21427))

*Previous behavior*:

```ipython
In [2]: ts = pd.Timestamp('2018-06-11 18:01:14')

In [3]: ts
Out[3]: Timestamp('2018-06-11 18:01:14')

In [4]: tic = pd.offsets.Hour(n=2, normalize=True)
   ...:

In [5]: tic
Out[5]: <2 * Hours>

In [6]: ts + tic
Out[6]: Timestamp('2018-06-11 00:00:00')

In [7]: ts + tic + tic + tic == ts + (tic + tic + tic)
Out[7]: False
```


*New behavior*:


<a id='whatsnew-0240-api-datetimelike'></a>

<a id='whatsnew-0240-api-period-subtraction'></a>

### Period subtraction

Subtraction of a `Period` from another `Period` will give a `DateOffset`.
instead of an integer ([GH21314](https://github.com/pandas-dev/pandas/issues/21314))

*Previous behavior*:

```ipython
In [2]: june = pd.Period('June 2018')

In [3]: april = pd.Period('April 2018')

In [4]: june - april
Out [4]: 2
```


*New behavior*:

Similarly, subtraction of a `Period` from a `PeriodIndex` will now return
an `Index` of `DateOffset` objects instead of an `Int64Index`

*Previous behavior*:

```ipython
In [2]: pi = pd.period_range('June 2018', freq='M', periods=3)

In [3]: pi - pi[0]
Out[3]: Int64Index([0, 1, 2], dtype='int64')
```


*New behavior*:


<a id='whatsnew-0240-api-timedelta64-subtract-nan'></a>

### Addition/subtraction of `NaN` from `DataFrame`

Adding or subtracting `NaN` from a `DataFrame` column with
`timedelta64[ns]` dtype will now raise a `TypeError` instead of returning
all-`NaT`.  This is for compatibility with `TimedeltaIndex` and
`Series` behavior ([GH22163](https://github.com/pandas-dev/pandas/issues/22163))

*Previous behavior*:

```ipython
In [4]: df = pd.DataFrame([pd.Timedelta(days=1)])

In [5]: df - np.nan
Out[5]:
    0
0 NaT
```


*New behavior*:

```ipython
In [2]: df - np.nan
...
TypeError: unsupported operand type(s) for -: 'TimedeltaIndex' and 'float'
```



<a id='whatsnew-0240-api-dataframe-cmp-broadcasting'></a>

### DataFrame comparison operations broadcasting changes

Previously, the broadcasting behavior of `DataFrame` comparison
operations (`==`, `!=`, …) was inconsistent with the behavior of
arithmetic operations (`+`, `-`, …).  The behavior of the comparison
operations has been changed to match the arithmetic operations in these cases.
([GH22880](https://github.com/pandas-dev/pandas/issues/22880))

The affected cases are:

- operating against a 2-dimensional `np.ndarray` with either 1 row or 1 column will now broadcast the same way a `np.ndarray` would ([GH23000](https://github.com/pandas-dev/pandas/issues/23000)).  
- a list or tuple with length matching the number of rows in the `DataFrame` will now raise `ValueError` instead of operating column-by-column ([GH22880](https://github.com/pandas-dev/pandas/issues/22880).  
- a list or tuple with length matching the number of columns in the `DataFrame` will now operate row-by-row instead of raising `ValueError` ([GH22880](https://github.com/pandas-dev/pandas/issues/22880)).  


*Previous behavior*:

```ipython
In [5]: df == arr[[0], :]
    ...: # comparison previously broadcast where arithmetic would raise
Out[5]:
       0      1
0   True   True
1  False  False
2  False  False
In [6]: df + arr[[0], :]
...
ValueError: Unable to coerce to DataFrame, shape must be (3, 2): given (1, 2)

In [7]: df == (1, 2)
    ...: # length matches number of columns;
    ...: # comparison previously raised where arithmetic would broadcast
...
ValueError: Invalid broadcasting comparison [(1, 2)] with block values
In [8]: df + (1, 2)
Out[8]:
   0  1
0  1  3
1  3  5
2  5  7

In [9]: df == (1, 2, 3)
    ...:  # length matches number of rows
    ...:  # comparison previously broadcast where arithmetic would raise
Out[9]:
       0      1
0  False   True
1   True  False
2  False  False
In [10]: df + (1, 2, 3)
...
ValueError: Unable to coerce to Series, length must be 2: given 3
```


*New behavior*:

```ipython
# Comparison operations and arithmetic operations both raise ValueError.
In [6]: df == (1, 2, 3)
...
ValueError: Unable to coerce to Series, length must be 2: given 3

In [7]: df + (1, 2, 3)
...
ValueError: Unable to coerce to Series, length must be 2: given 3
```



<a id='whatsnew-0240-api-dataframe-arithmetic-broadcasting'></a>

### DataFrame arithmetic operations broadcasting changes

`DataFrame` arithmetic operations when operating with 2-dimensional
`np.ndarray` objects now broadcast in the same way as `np.ndarray`
broadcast.  ([GH23000](https://github.com/pandas-dev/pandas/issues/23000))

*Previous behavior*:

```ipython
In [5]: df + arr[[0], :]   # 1 row, 2 columns
...
ValueError: Unable to coerce to DataFrame, shape must be (3, 2): given (1, 2)
In [6]: df + arr[:, [1]]   # 1 column, 3 rows
...
ValueError: Unable to coerce to DataFrame, shape must be (3, 2): given (3, 1)
```


*New behavior*:


<a id='whatsnew-0240-api-incompatibilities'></a>

### Series and Index data-dtype incompatibilities

`Series` and `Index` constructors now raise when the
data is incompatible with a passed `dtype=` ([GH15832](https://github.com/pandas-dev/pandas/issues/15832))

*Previous behavior*:

```ipython
In [4]: pd.Series([-1], dtype="uint64")
Out [4]:
0    18446744073709551615
dtype: uint64
```


*New behavior*:

```ipython
In [4]: pd.Series([-1], dtype="uint64")
Out [4]:
...
OverflowError: Trying to coerce negative values to unsigned integers
```



<a id='whatsnew-0240-api-concat-categorical'></a>

### Concatenation Changes

Calling `pandas.concat()` on a `Categorical` of ints with NA values now
causes them to be processed as objects when concatenating with anything
other than another `Categorical` of ints ([GH19214](https://github.com/pandas-dev/pandas/issues/19214))

*Previous behavior*

```ipython
In [3]: pd.concat([s, c])
Out[3]:
0    0.0
1    1.0
2    NaN
0    0.0
1    1.0
2    NaN
dtype: float64
```


*New behavior*

### Datetimelike API changes

- For `DatetimeIndex` and `TimedeltaIndex` with non-`None` `freq` attribute, addition or subtraction of integer-dtyped array or `Index` will return an object of the same class ([GH19959](https://github.com/pandas-dev/pandas/issues/19959))  
- `DateOffset` objects are now immutable. Attempting to alter one of these will now raise `AttributeError` ([GH21341](https://github.com/pandas-dev/pandas/issues/21341))  
- `PeriodIndex` subtraction of another `PeriodIndex` will now return an object-dtype `Index` of `DateOffset` objects instead of raising a `TypeError` ([GH20049](https://github.com/pandas-dev/pandas/issues/20049))  
- `cut()` and `qcut()` now returns a `DatetimeIndex` or `TimedeltaIndex` bins when the input is datetime or timedelta dtype respectively and `retbins=True` ([GH19891](https://github.com/pandas-dev/pandas/issues/19891))  
- `DatetimeIndex.to_period()` and `Timestamp.to_period()` will issue a warning when timezone information will be lost ([GH21333](https://github.com/pandas-dev/pandas/issues/21333))  
- `PeriodIndex.tz_convert()` and `PeriodIndex.tz_localize()` have been removed ([GH21781](https://github.com/pandas-dev/pandas/issues/21781))  



<a id='whatsnew-0240-api-other'></a>

### Other API changes

- A newly constructed empty `DataFrame` with integer as the `dtype` will now only be cast to `float64` if `index` is specified ([GH22858](https://github.com/pandas-dev/pandas/issues/22858))  
- `Series.str.cat()` will now raise if `others` is a `set` ([GH23009](https://github.com/pandas-dev/pandas/issues/23009))  
- Passing scalar values to `DatetimeIndex` or `TimedeltaIndex` will now raise `TypeError` instead of `ValueError` ([GH23539](https://github.com/pandas-dev/pandas/issues/23539))  
- `max_rows` and `max_cols` parameters removed from `HTMLFormatter` since truncation is handled by `DataFrameFormatter` ([GH23818](https://github.com/pandas-dev/pandas/issues/23818))  
- `read_csv()` will now raise a `ValueError` if a column with missing values is declared as having dtype `bool` ([GH20591](https://github.com/pandas-dev/pandas/issues/20591))  
- The column order of the resultant `DataFrame` from `MultiIndex.to_frame()` is now guaranteed to match the `MultiIndex.names` order. ([GH22420](https://github.com/pandas-dev/pandas/issues/22420))  
- Incorrectly passing a `DatetimeIndex` to `MultiIndex.from_tuples()`, rather than a sequence of tuples, now raises a `TypeError` rather than a `ValueError` ([GH24024](https://github.com/pandas-dev/pandas/issues/24024))  
- `pd.offsets.generate_range()` argument `time_rule` has been removed; use `offset` instead ([GH24157](https://github.com/pandas-dev/pandas/issues/24157))  
- In 0.23.x, pandas would raise a `ValueError` on a merge of a numeric column (e.g. `int` dtyped column) and an `object` dtyped column ([GH9780](https://github.com/pandas-dev/pandas/issues/9780)). We have re-enabled the ability to merge `object` and other dtypes; pandas will still raise on a merge between a numeric and an `object` dtyped column that is composed only of strings ([GH21681](https://github.com/pandas-dev/pandas/issues/21681))  
- Accessing a level of a `MultiIndex` with a duplicate name (e.g. in
  `get_level_values()`) now raises a `ValueError` instead of a `KeyError` ([GH21678](https://github.com/pandas-dev/pandas/issues/21678)).  
- Invalid construction of `IntervalDtype` will now always raise a `TypeError` rather than a `ValueError` if the subdtype is invalid ([GH21185](https://github.com/pandas-dev/pandas/issues/21185))  
- Trying to reindex a `DataFrame` with a non unique `MultiIndex` now raises a `ValueError` instead of an `Exception` ([GH21770](https://github.com/pandas-dev/pandas/issues/21770))  
- `Index` subtraction will attempt to operate element-wise instead of raising `TypeError` ([GH19369](https://github.com/pandas-dev/pandas/issues/19369))  
- `pandas.io.formats.style.Styler` supports a `number-format` property when using `to_excel()` ([GH22015](https://github.com/pandas-dev/pandas/issues/22015))  
- `DataFrame.corr()` and `Series.corr()` now raise a `ValueError` along with a helpful error message instead of a `KeyError` when supplied with an invalid method ([GH22298](https://github.com/pandas-dev/pandas/issues/22298))  
- `shift()` will now always return a copy, instead of the previous behaviour of returning self when shifting by 0 ([GH22397](https://github.com/pandas-dev/pandas/issues/22397))  
- `DataFrame.set_index()` now gives a better (and less frequent) KeyError, raises a `ValueError` for incorrect types,
  and will not fail on duplicate column names with `drop=True`. ([GH22484](https://github.com/pandas-dev/pandas/issues/22484))  
- Slicing a single row of a DataFrame with multiple ExtensionArrays of the same type now preserves the dtype, rather than coercing to object ([GH22784](https://github.com/pandas-dev/pandas/issues/22784))  
- `DateOffset` attribute _cacheable and method _should_cache have been removed ([GH23118](https://github.com/pandas-dev/pandas/issues/23118))  
- `Series.searchsorted()`, when supplied a scalar value to search for, now returns a scalar instead of an array ([GH23801](https://github.com/pandas-dev/pandas/issues/23801)).  
- `Categorical.searchsorted()`, when supplied a scalar value to search for, now returns a scalar instead of an array ([GH23466](https://github.com/pandas-dev/pandas/issues/23466)).  
- `Categorical.searchsorted()` now raises a `KeyError` rather that a `ValueError`, if a searched for key is not found in its categories ([GH23466](https://github.com/pandas-dev/pandas/issues/23466)).  
- `Index.hasnans()` and `Series.hasnans()` now always return a python boolean. Previously, a python or a numpy boolean could be returned, depending on circumstances ([GH23294](https://github.com/pandas-dev/pandas/issues/23294)).  
- The order of the arguments of `DataFrame.to_html()` and `DataFrame.to_string()` is rearranged to be consistent with each other. ([GH23614](https://github.com/pandas-dev/pandas/issues/23614))  
- `CategoricalIndex.reindex()` now raises a `ValueError` if the target index is non-unique and not equal to the current index. It previously only raised if the target index was not of a categorical dtype ([GH23963](https://github.com/pandas-dev/pandas/issues/23963)).  
- `Series.to_list()` and `Index.to_list()` are now aliases of `Series.tolist` respectively `Index.tolist` ([GH8826](https://github.com/pandas-dev/pandas/issues/8826))  
- The result of `SparseSeries.unstack` is now a `DataFrame` with sparse values, rather than a `SparseDataFrame` ([GH24372](https://github.com/pandas-dev/pandas/issues/24372)).  
- `DatetimeIndex` and `TimedeltaIndex` no longer ignore the dtype precision. Passing a non-nanosecond resolution dtype will raise a `ValueError` ([GH24753](https://github.com/pandas-dev/pandas/issues/24753))  



<a id='whatsnew-0240-api-extension'></a>

## Extension type changes

**Equality and hashability**

Pandas now requires that extension dtypes be hashable (i.e. the respective
`ExtensionDtype` objects; hashability is not a requirement for the values
of the corresponding `ExtensionArray`). The base class implements
a default `__eq__` and `__hash__`. If you have a parametrized dtype, you should
update the `ExtensionDtype._metadata` tuple to match the signature of your
`__init__` method. See `pandas.api.extensions.ExtensionDtype` for more ([GH22476](https://github.com/pandas-dev/pandas/issues/22476)).

**New and changed methods**

- `dropna()` has been added ([GH21185](https://github.com/pandas-dev/pandas/issues/21185))  
- `repeat()` has been added ([GH24349](https://github.com/pandas-dev/pandas/issues/24349))  
- The `ExtensionArray` constructor, `_from_sequence` now take the keyword arg `copy=False` ([GH21185](https://github.com/pandas-dev/pandas/issues/21185))  
- `pandas.api.extensions.ExtensionArray.shift()` added as part of the basic `ExtensionArray` interface ([GH22387](https://github.com/pandas-dev/pandas/issues/22387)).  
- `searchsorted()` has been added ([GH24350](https://github.com/pandas-dev/pandas/issues/24350))  
- Support for reduction operations such as `sum`, `mean` via opt-in base class method override ([GH22762](https://github.com/pandas-dev/pandas/issues/22762))  
- `ExtensionArray.isna()` is allowed to return an `ExtensionArray` ([GH22325](https://github.com/pandas-dev/pandas/issues/22325)).  


**Dtype changes**

- `ExtensionDtype` has gained the ability to instantiate from string dtypes, e.g. `decimal` would instantiate a registered `DecimalDtype`; furthermore
  the `ExtensionDtype` has gained the method `construct_array_type` ([GH21185](https://github.com/pandas-dev/pandas/issues/21185))  
- Added `ExtensionDtype._is_numeric` for controlling whether an extension dtype is considered numeric ([GH22290](https://github.com/pandas-dev/pandas/issues/22290)).  
- Added `pandas.api.types.register_extension_dtype()` to register an extension type with pandas ([GH22664](https://github.com/pandas-dev/pandas/issues/22664))  
- Updated the `.type` attribute for `PeriodDtype`, `DatetimeTZDtype`, and `IntervalDtype` to be instances of the dtype (`Period`, `Timestamp`, and `Interval` respectively) ([GH22938](https://github.com/pandas-dev/pandas/issues/22938))  



<a id='whatsnew-0240-enhancements-extension-array-operators'></a>
**Operator support**

A `Series` based on an `ExtensionArray` now supports arithmetic and comparison
operators ([GH19577](https://github.com/pandas-dev/pandas/issues/19577)). There are two approaches for providing operator support for an `ExtensionArray`:

1. Define each of the operators on your `ExtensionArray` subclass.  
1. Use an operator implementation from pandas that depends on operators that are already defined
  on the underlying elements (scalars) of the `ExtensionArray`.  


See the [ExtensionArray Operator Support](development/extending.ipynb#extending-extension-operator) documentation section for details on both
ways of adding operator support.

**Other changes**

- A default repr for `pandas.api.extensions.ExtensionArray` is now provided ([GH23601](https://github.com/pandas-dev/pandas/issues/23601)).  
- `ExtensionArray._formatting_values()` is deprecated. Use `ExtensionArray._formatter` instead. ([GH23601](https://github.com/pandas-dev/pandas/issues/23601))  
- An `ExtensionArray` with a boolean dtype now works correctly as a boolean indexer. `pandas.api.types.is_bool_dtype()` now properly considers them boolean ([GH22326](https://github.com/pandas-dev/pandas/issues/22326))  


**Bug fixes**

- Bug in `Series.get()` for `Series` using `ExtensionArray` and integer index ([GH21257](https://github.com/pandas-dev/pandas/issues/21257))  
- `shift()` now dispatches to `ExtensionArray.shift()` ([GH22386](https://github.com/pandas-dev/pandas/issues/22386))  
- `Series.combine()` works correctly with `ExtensionArray` inside of `Series` ([GH20825](https://github.com/pandas-dev/pandas/issues/20825))  
- `Series.combine()` with scalar argument now works for any function type ([GH21248](https://github.com/pandas-dev/pandas/issues/21248))  
- `Series.astype()` and `DataFrame.astype()` now dispatch to `ExtensionArray.astype()` ([GH21185](https://github.com/pandas-dev/pandas/issues/21185)).  
- Slicing a single row of a `DataFrame` with multiple ExtensionArrays of the same type now preserves the dtype, rather than coercing to object ([GH22784](https://github.com/pandas-dev/pandas/issues/22784))  
- Bug when concatenating multiple `Series` with different extension dtypes not casting to object dtype ([GH22994](https://github.com/pandas-dev/pandas/issues/22994))  
- Series backed by an `ExtensionArray` now work with `util.hash_pandas_object()` ([GH23066](https://github.com/pandas-dev/pandas/issues/23066))  
- `DataFrame.stack()` no longer converts to object dtype for DataFrames where each column has the same extension dtype. The output Series will have the same dtype as the columns ([GH23077](https://github.com/pandas-dev/pandas/issues/23077)).  
- `Series.unstack()` and `DataFrame.unstack()` no longer convert extension arrays to object-dtype ndarrays. Each column in the output `DataFrame` will now have the same dtype as the input ([GH23077](https://github.com/pandas-dev/pandas/issues/23077)).  
- Bug when grouping `Dataframe.groupby()` and aggregating on `ExtensionArray` it was not returning the actual `ExtensionArray` dtype ([GH23227](https://github.com/pandas-dev/pandas/issues/23227)).  
- Bug in `pandas.merge()` when merging on an extension array-backed column ([GH23020](https://github.com/pandas-dev/pandas/issues/23020)).  



<a id='whatsnew-0240-deprecations'></a>

## Deprecations

- `MultiIndex.labels` has been deprecated and replaced by `MultiIndex.codes`.
  The functionality is unchanged. The new name better reflects the natures of
  these codes and makes the `MultiIndex` API more similar to the API for `CategoricalIndex` ([GH13443](https://github.com/pandas-dev/pandas/issues/13443)).
  As a consequence, other uses of the name `labels` in `MultiIndex` have also been deprecated and replaced with `codes`:  
  - You should initialize a `MultiIndex` instance using a parameter named `codes` rather than `labels`.  
  - `MultiIndex.set_labels` has been deprecated in favor of `MultiIndex.set_codes()`.  
  - For method `MultiIndex.copy()`, the `labels` parameter has been deprecated and replaced by a `codes` parameter.  
- `DataFrame.to_stata()`, `read_stata()`, `StataReader` and `StataWriter` have deprecated the `encoding` argument. The encoding of a Stata dta file is determined by the file type and cannot be changed ([GH21244](https://github.com/pandas-dev/pandas/issues/21244))  
- `MultiIndex.to_hierarchical()` is deprecated and will be removed in a future version ([GH21613](https://github.com/pandas-dev/pandas/issues/21613))  
- `Series.ptp()` is deprecated. Use `numpy.ptp` instead ([GH21614](https://github.com/pandas-dev/pandas/issues/21614))  
- `Series.compress()` is deprecated. Use `Series[condition]` instead ([GH18262](https://github.com/pandas-dev/pandas/issues/18262))  
- The signature of `Series.to_csv()` has been uniformed to that of `DataFrame.to_csv()`: the name of the first argument is now `path_or_buf`, the order of subsequent arguments has changed, the `header` argument now defaults to `True`. ([GH19715](https://github.com/pandas-dev/pandas/issues/19715))  
- `Categorical.from_codes()` has deprecated providing float values for the `codes` argument. ([GH21767](https://github.com/pandas-dev/pandas/issues/21767))  
- `pandas.read_table()` is deprecated. Instead, use `read_csv()` passing `sep='\t'` if necessary. This deprecation has been removed in 0.25.0. ([GH21948](https://github.com/pandas-dev/pandas/issues/21948))  
- `Series.str.cat()` has deprecated using arbitrary list-likes *within* list-likes. A list-like container may still contain
  many `Series`, `Index` or 1-dimensional `np.ndarray`, or alternatively, only scalar values. ([GH21950](https://github.com/pandas-dev/pandas/issues/21950))  
- `FrozenNDArray.searchsorted()` has deprecated the `v` parameter in favor of `value` ([GH14645](https://github.com/pandas-dev/pandas/issues/14645))  
- `DatetimeIndex.shift()` and `PeriodIndex.shift()` now accept `periods` argument instead of `n` for consistency with `Index.shift()` and `Series.shift()`. Using `n` throws a deprecation warning ([GH22458](https://github.com/pandas-dev/pandas/issues/22458), [GH22912](https://github.com/pandas-dev/pandas/issues/22912))  
- The `fastpath` keyword of the different Index constructors is deprecated ([GH23110](https://github.com/pandas-dev/pandas/issues/23110)).  
- `Timestamp.tz_localize()`, `DatetimeIndex.tz_localize()`, and `Series.tz_localize()` have deprecated the `errors` argument in favor of the `nonexistent` argument ([GH8917](https://github.com/pandas-dev/pandas/issues/8917))  
- The class `FrozenNDArray` has been deprecated. When unpickling, `FrozenNDArray` will be unpickled to `np.ndarray` once this class is removed ([GH9031](https://github.com/pandas-dev/pandas/issues/9031))  
- The methods `DataFrame.update()` and `Panel.update()` have deprecated the `raise_conflict=False|True` keyword in favor of `errors='ignore'|'raise'` ([GH23585](https://github.com/pandas-dev/pandas/issues/23585))  
- The methods `Series.str.partition()` and `Series.str.rpartition()` have deprecated the `pat` keyword in favor of `sep` ([GH22676](https://github.com/pandas-dev/pandas/issues/22676))  
- Deprecated the `nthreads` keyword of `pandas.read_feather()` in favor of `use_threads` to reflect the changes in `pyarrow>=0.11.0`. ([GH23053](https://github.com/pandas-dev/pandas/issues/23053))  
- `pandas.read_excel()` has deprecated accepting `usecols` as an integer. Please pass in a list of ints from 0 to `usecols` inclusive instead ([GH23527](https://github.com/pandas-dev/pandas/issues/23527))  
- Constructing a `TimedeltaIndex` from data with `datetime64`-dtyped data is deprecated, will raise `TypeError` in a future version ([GH23539](https://github.com/pandas-dev/pandas/issues/23539))  
- Constructing a `DatetimeIndex` from data with `timedelta64`-dtyped data is deprecated, will raise `TypeError` in a future version ([GH23675](https://github.com/pandas-dev/pandas/issues/23675))  
- The `keep_tz=False` option (the default) of the `keep_tz` keyword of
  `DatetimeIndex.to_series()` is deprecated ([GH17832](https://github.com/pandas-dev/pandas/issues/17832)).  
- Timezone converting a tz-aware `datetime.datetime` or `Timestamp` with `Timestamp` and the `tz` argument is now deprecated. Instead, use `Timestamp.tz_convert()` ([GH23579](https://github.com/pandas-dev/pandas/issues/23579))  
- `pandas.api.types.is_period()` is deprecated in favor of `pandas.api.types.is_period_dtype` ([GH23917](https://github.com/pandas-dev/pandas/issues/23917))  
- `pandas.api.types.is_datetimetz()` is deprecated in favor of `pandas.api.types.is_datetime64tz` ([GH23917](https://github.com/pandas-dev/pandas/issues/23917))  
- Creating a `TimedeltaIndex`, `DatetimeIndex`, or `PeriodIndex` by passing range arguments start, end, and periods is deprecated in favor of `timedelta_range()`, `date_range()`, or `period_range()` ([GH23919](https://github.com/pandas-dev/pandas/issues/23919))  
- Passing a string alias like `'datetime64[ns, UTC]'` as the `unit` parameter to `DatetimeTZDtype` is deprecated. Use `DatetimeTZDtype.construct_from_string` instead ([GH23990](https://github.com/pandas-dev/pandas/issues/23990)).  
- The `skipna` parameter of `infer_dtype()` will switch to `True` by default in a future version of pandas ([GH17066](https://github.com/pandas-dev/pandas/issues/17066), [GH24050](https://github.com/pandas-dev/pandas/issues/24050))  
- In `Series.where()` with Categorical data, providing an `other` that is not present in the categories is deprecated. Convert the categorical to a different dtype or add the `other` to the categories first ([GH24077](https://github.com/pandas-dev/pandas/issues/24077)).  
- `Series.clip_lower()`, `Series.clip_upper()`, `DataFrame.clip_lower()` and `DataFrame.clip_upper()` are deprecated and will be removed in a future version. Use `Series.clip(lower=threshold)`, `Series.clip(upper=threshold)` and the equivalent `DataFrame` methods ([GH24203](https://github.com/pandas-dev/pandas/issues/24203))  
- `Series.nonzero()` is deprecated and will be removed in a future version ([GH18262](https://github.com/pandas-dev/pandas/issues/18262))  
- Passing an integer to `Series.fillna()` and `DataFrame.fillna()` with `timedelta64[ns]` dtypes is deprecated, will raise `TypeError` in a future version.  Use `obj.fillna(pd.Timedelta(...))` instead ([GH24694](https://github.com/pandas-dev/pandas/issues/24694))  
- `Series.cat.categorical`, `Series.cat.name` and `Series.cat.index` have been deprecated. Use the attributes on `Series.cat` or `Series` directly. ([GH24751](https://github.com/pandas-dev/pandas/issues/24751)).  
- Passing a dtype without a precision like `np.dtype('datetime64')` or `timedelta64` to `Index`, `DatetimeIndex` and `TimedeltaIndex` is now deprecated. Use the nanosecond-precision dtype instead ([GH24753](https://github.com/pandas-dev/pandas/issues/24753)).  



<a id='whatsnew-0240-deprecations-datetimelike-int-ops'></a>

### Integer addition/subtraction with datetimes and timedeltas is deprecated

In the past, users could—in some cases—add or subtract integers or integer-dtype
arrays from `Timestamp`, `DatetimeIndex` and `TimedeltaIndex`.

This usage is now deprecated.  Instead add or subtract integer multiples of
the object’s `freq` attribute ([GH21939](https://github.com/pandas-dev/pandas/issues/21939), [GH23878](https://github.com/pandas-dev/pandas/issues/23878)).

*Previous behavior*:

```ipython
In [5]: ts = pd.Timestamp('1994-05-06 12:15:16', freq=pd.offsets.Hour())
In [6]: ts + 2
Out[6]: Timestamp('1994-05-06 14:15:16', freq='H')

In [7]: tdi = pd.timedelta_range('1D', periods=2)
In [8]: tdi - np.array([2, 1])
Out[8]: TimedeltaIndex(['-1 days', '1 days'], dtype='timedelta64[ns]', freq=None)

In [9]: dti = pd.date_range('2001-01-01', periods=2, freq='7D')
In [10]: dti + pd.Index([1, 2])
Out[10]: DatetimeIndex(['2001-01-08', '2001-01-22'], dtype='datetime64[ns]', freq=None)
```


*New behavior*:


<a id='whatsnew-0240-deprecations-integer-tz'></a>

### Passing integer data and a timezone to datetimeindex

The behavior of `DatetimeIndex` when passed integer data and
a timezone is changing in a future version of pandas. Previously, these
were interpreted as wall times in the desired timezone. In the future,
these will be interpreted as wall times in UTC, which are then converted
to the desired timezone ([GH24559](https://github.com/pandas-dev/pandas/issues/24559)).

The default behavior remains the same, but issues a warning:

```ipython
In [3]: pd.DatetimeIndex([946684800000000000], tz="US/Central")
/bin/ipython:1: FutureWarning:
    Passing integer-dtype data and a timezone to DatetimeIndex. Integer values
    will be interpreted differently in a future version of pandas. Previously,
    these were viewed as datetime64[ns] values representing the wall time
    *in the specified timezone*. In the future, these will be viewed as
    datetime64[ns] values representing the wall time *in UTC*. This is similar
    to a nanosecond-precision UNIX epoch. To accept the future behavior, use

        pd.to_datetime(integer_data, utc=True).tz_convert(tz)

    To keep the previous behavior, use

        pd.to_datetime(integer_data).tz_localize(tz)

 #!/bin/python3
 Out[3]: DatetimeIndex(['2000-01-01 00:00:00-06:00'], dtype='datetime64[ns, US/Central]', freq=None)
```


As the warning message explains, opt in to the future behavior by specifying that
the integer values are UTC, and then converting to the final timezone:

The old behavior can be retained with by localizing directly to the final timezone:


<a id='whatsnew-0240-deprecations-tz-aware-array'></a>

### Converting timezone-aware Series and Index to NumPy arrays

The conversion from a `Series` or `Index` with timezone-aware
datetime data will change to preserve timezones by default ([GH23569](https://github.com/pandas-dev/pandas/issues/23569)).

NumPy doesn’t have a dedicated dtype for timezone-aware datetimes.
In the past, converting a `Series` or `DatetimeIndex` with
timezone-aware datatimes would convert to a NumPy array by

1. converting the tz-aware data to UTC  
1. dropping the timezone-info  
1. returning a [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) with `datetime64[ns]` dtype  


Future versions of pandas will preserve the timezone information by returning an
object-dtype NumPy array where each value is a `Timestamp` with the correct
timezone attached

The default behavior remains the same, but issues a warning

In [ ]:
In [8]: np.asarray(ser)
/bin/ipython:1: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive
      ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray
      with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.

        To accept the future behavior, pass 'dtype=object'.
        To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  #!/bin/python3
Out[8]:
array(['1999-12-31T23:00:00.000000000', '2000-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')

The previous or future behavior can be obtained, without any warnings, by specifying
the `dtype`

*Previous behavior*

*Future behavior*

Or by using `Series.to_numpy()`

All the above applies to a `DatetimeIndex` with tz-aware values as well.


<a id='whatsnew-0240-prior-deprecations'></a>

## Removal of prior version deprecations/changes

- The `LongPanel` and `WidePanel` classes have been removed ([GH10892](https://github.com/pandas-dev/pandas/issues/10892))  
- `Series.repeat()` has renamed the `reps` argument to `repeats` ([GH14645](https://github.com/pandas-dev/pandas/issues/14645))  
- Several private functions were removed from the (non-public) module `pandas.core.common` ([GH22001](https://github.com/pandas-dev/pandas/issues/22001))  
- Removal of the previously deprecated module `pandas.core.datetools` ([GH14105](https://github.com/pandas-dev/pandas/issues/14105), [GH14094](https://github.com/pandas-dev/pandas/issues/14094))  
- Strings passed into `DataFrame.groupby()` that refer to both column and index levels will raise a `ValueError` ([GH14432](https://github.com/pandas-dev/pandas/issues/14432))  
- `Index.repeat()` and `MultiIndex.repeat()` have renamed the `n` argument to `repeats` ([GH14645](https://github.com/pandas-dev/pandas/issues/14645))  
- The `Series` constructor and `.astype` method will now raise a `ValueError` if timestamp dtypes are passed in without a unit (e.g. `np.datetime64`) for the `dtype` parameter ([GH15987](https://github.com/pandas-dev/pandas/issues/15987))  
- Removal of the previously deprecated `as_indexer` keyword completely from `str.match()` ([GH22356](https://github.com/pandas-dev/pandas/issues/22356), [GH6581](https://github.com/pandas-dev/pandas/issues/6581))  
- The modules `pandas.types`, `pandas.computation`, and `pandas.util.decorators` have been removed ([GH16157](https://github.com/pandas-dev/pandas/issues/16157), [GH16250](https://github.com/pandas-dev/pandas/issues/16250))  
- Removed the `pandas.formats.style` shim for `pandas.io.formats.style.Styler` ([GH16059](https://github.com/pandas-dev/pandas/issues/16059))  
- `pandas.pnow`, `pandas.match`, `pandas.groupby`, `pd.get_store`, `pd.Expr`, and `pd.Term` have been removed ([GH15538](https://github.com/pandas-dev/pandas/issues/15538), [GH15940](https://github.com/pandas-dev/pandas/issues/15940))  
- `Categorical.searchsorted()` and `Series.searchsorted()` have renamed the `v` argument to `value` ([GH14645](https://github.com/pandas-dev/pandas/issues/14645))  
- `pandas.parser`, `pandas.lib`, and `pandas.tslib` have been removed ([GH15537](https://github.com/pandas-dev/pandas/issues/15537))  
- `Index.searchsorted()` have renamed the `key` argument to `value` ([GH14645](https://github.com/pandas-dev/pandas/issues/14645))  
- `DataFrame.consolidate` and `Series.consolidate` have been removed ([GH15501](https://github.com/pandas-dev/pandas/issues/15501))  
- Removal of the previously deprecated module `pandas.json` ([GH19944](https://github.com/pandas-dev/pandas/issues/19944))  
- The module `pandas.tools` has been removed ([GH15358](https://github.com/pandas-dev/pandas/issues/15358), [GH16005](https://github.com/pandas-dev/pandas/issues/16005))  
- `SparseArray.get_values()` and `SparseArray.to_dense()` have dropped the `fill` parameter ([GH14686](https://github.com/pandas-dev/pandas/issues/14686))  
- `DataFrame.sortlevel` and `Series.sortlevel` have been removed ([GH15099](https://github.com/pandas-dev/pandas/issues/15099))  
- `SparseSeries.to_dense()` has dropped the `sparse_only` parameter ([GH14686](https://github.com/pandas-dev/pandas/issues/14686))  
- `DataFrame.astype()` and `Series.astype()` have renamed the `raise_on_error` argument to `errors` ([GH14967](https://github.com/pandas-dev/pandas/issues/14967))  
- `is_sequence`, `is_any_int_dtype`, and `is_floating_dtype` have been removed from `pandas.api.types` ([GH16163](https://github.com/pandas-dev/pandas/issues/16163), [GH16189](https://github.com/pandas-dev/pandas/issues/16189))  



<a id='whatsnew-0240-performance'></a>

## Performance improvements

- Slicing Series and DataFrames with an monotonically increasing `CategoricalIndex`
  is now very fast and has speed comparable to slicing with an `Int64Index`.
  The speed increase is both when indexing by label (using .loc) and position(.iloc) ([GH20395](https://github.com/pandas-dev/pandas/issues/20395))
  Slicing a monotonically increasing `CategoricalIndex` itself (i.e. `ci[1000:2000]`)
  shows similar speed improvements as above ([GH21659](https://github.com/pandas-dev/pandas/issues/21659))  
- Improved performance of `CategoricalIndex.equals()` when comparing to another `CategoricalIndex` ([GH24023](https://github.com/pandas-dev/pandas/issues/24023))  
- Improved performance of `Series.describe()` in case of numeric dtpyes ([GH21274](https://github.com/pandas-dev/pandas/issues/21274))  
- Improved performance of `pandas.core.groupby.GroupBy.rank()` when dealing with tied rankings ([GH21237](https://github.com/pandas-dev/pandas/issues/21237))  
- Improved performance of `DataFrame.set_index()` with columns consisting of `Period` objects ([GH21582](https://github.com/pandas-dev/pandas/issues/21582), [GH21606](https://github.com/pandas-dev/pandas/issues/21606))  
- Improved performance of `Series.at()` and `Index.get_value()` for Extension Arrays values (e.g. `Categorical`) ([GH24204](https://github.com/pandas-dev/pandas/issues/24204))  
- Improved performance of membership checks in `Categorical` and `CategoricalIndex`
  (i.e. `x in cat`-style checks are much faster). `CategoricalIndex.contains()`
  is likewise much faster ([GH21369](https://github.com/pandas-dev/pandas/issues/21369), [GH21508](https://github.com/pandas-dev/pandas/issues/21508))  
- Improved performance of `HDFStore.groups()` (and dependent functions like
  `HDFStore.keys()`.  (i.e. `x in store` checks are much faster)
  ([GH21372](https://github.com/pandas-dev/pandas/issues/21372))  
- Improved the performance of `pandas.get_dummies()` with `sparse=True` ([GH21997](https://github.com/pandas-dev/pandas/issues/21997))  
- Improved performance of `IndexEngine.get_indexer_non_unique()` for sorted, non-unique indexes ([GH9466](https://github.com/pandas-dev/pandas/issues/9466))  
- Improved performance of `PeriodIndex.unique()` ([GH23083](https://github.com/pandas-dev/pandas/issues/23083))  
- Improved performance of `concat()` for Series objects ([GH23404](https://github.com/pandas-dev/pandas/issues/23404))  
- Improved performance of `DatetimeIndex.normalize()` and `Timestamp.normalize()` for timezone naive or UTC datetimes ([GH23634](https://github.com/pandas-dev/pandas/issues/23634))  
- Improved performance of `DatetimeIndex.tz_localize()` and various `DatetimeIndex` attributes with dateutil UTC timezone ([GH23772](https://github.com/pandas-dev/pandas/issues/23772))  
- Fixed a performance regression on Windows with Python 3.7 of `read_csv()` ([GH23516](https://github.com/pandas-dev/pandas/issues/23516))  
- Improved performance of `Categorical` constructor for `Series` objects ([GH23814](https://github.com/pandas-dev/pandas/issues/23814))  
- Improved performance of `where()` for Categorical data ([GH24077](https://github.com/pandas-dev/pandas/issues/24077))  
- Improved performance of iterating over a `Series`. Using `DataFrame.itertuples()` now creates iterators
  without internally allocating lists of all elements ([GH20783](https://github.com/pandas-dev/pandas/issues/20783))  
- Improved performance of `Period` constructor, additionally benefitting `PeriodArray` and `PeriodIndex` creation ([GH24084](https://github.com/pandas-dev/pandas/issues/24084), [GH24118](https://github.com/pandas-dev/pandas/issues/24118))  
- Improved performance of tz-aware `DatetimeArray` binary operations ([GH24491](https://github.com/pandas-dev/pandas/issues/24491))  



<a id='whatsnew-0240-bug-fixes'></a>

## Bug fixes

### Categorical

- Bug in `Categorical.from_codes()` where `NaN` values in `codes` were silently converted to `0` ([GH21767](https://github.com/pandas-dev/pandas/issues/21767)). In the future this will raise a `ValueError`. Also changes the behavior of `.from_codes([1.1, 2.0])`.  
- Bug in `Categorical.sort_values()` where `NaN` values were always positioned in front regardless of `na_position` value. ([GH22556](https://github.com/pandas-dev/pandas/issues/22556)).  
- Bug when indexing with a boolean-valued `Categorical`. Now a boolean-valued `Categorical` is treated as a boolean mask ([GH22665](https://github.com/pandas-dev/pandas/issues/22665))  
- Constructing a `CategoricalIndex` with empty values and boolean categories was raising a `ValueError` after a change to dtype coercion ([GH22702](https://github.com/pandas-dev/pandas/issues/22702)).  
- Bug in `Categorical.take()` with a user-provided `fill_value` not encoding the `fill_value`, which could result in a `ValueError`, incorrect results, or a segmentation fault ([GH23296](https://github.com/pandas-dev/pandas/issues/23296)).  
- In `Series.unstack()`, specifying a `fill_value` not present in the categories now raises a `TypeError` rather than ignoring the `fill_value` ([GH23284](https://github.com/pandas-dev/pandas/issues/23284))  
- Bug when resampling `DataFrame.resample()` and aggregating on categorical data, the categorical dtype was getting lost. ([GH23227](https://github.com/pandas-dev/pandas/issues/23227))  
- Bug in many methods of the `.str`-accessor, which always failed on calling the `CategoricalIndex.str` constructor ([GH23555](https://github.com/pandas-dev/pandas/issues/23555), [GH23556](https://github.com/pandas-dev/pandas/issues/23556))  
- Bug in `Series.where()` losing the categorical dtype for categorical data ([GH24077](https://github.com/pandas-dev/pandas/issues/24077))  
- Bug in `Categorical.apply()` where `NaN` values could be handled unpredictably. They now remain unchanged ([GH24241](https://github.com/pandas-dev/pandas/issues/24241))  
- Bug in `Categorical` comparison methods incorrectly raising `ValueError` when operating against a `DataFrame` ([GH24630](https://github.com/pandas-dev/pandas/issues/24630))  
- Bug in `Categorical.set_categories()` where setting fewer new categories with `rename=True` caused a segmentation fault ([GH24675](https://github.com/pandas-dev/pandas/issues/24675))  

### Datetimelike

- Fixed bug where two `DateOffset` objects with different `normalize` attributes could evaluate as equal ([GH21404](https://github.com/pandas-dev/pandas/issues/21404))  
- Fixed bug where `Timestamp.resolution()` incorrectly returned 1-microsecond `timedelta` instead of 1-nanosecond `Timedelta` ([GH21336](https://github.com/pandas-dev/pandas/issues/21336), [GH21365](https://github.com/pandas-dev/pandas/issues/21365))  
- Bug in `to_datetime()` that did not consistently return an `Index` when `box=True` was specified ([GH21864](https://github.com/pandas-dev/pandas/issues/21864))  
- Bug in `DatetimeIndex` comparisons where string comparisons incorrectly raises `TypeError` ([GH22074](https://github.com/pandas-dev/pandas/issues/22074))  
- Bug in `DatetimeIndex` comparisons when comparing against `timedelta64[ns]` dtyped arrays; in some cases `TypeError` was incorrectly raised, in others it incorrectly failed to raise ([GH22074](https://github.com/pandas-dev/pandas/issues/22074))  
- Bug in `DatetimeIndex` comparisons when comparing against object-dtyped arrays ([GH22074](https://github.com/pandas-dev/pandas/issues/22074))  
- Bug in `DataFrame` with `datetime64[ns]` dtype addition and subtraction with `Timedelta`-like objects ([GH22005](https://github.com/pandas-dev/pandas/issues/22005), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DataFrame` with `datetime64[ns]` dtype addition and subtraction with `DateOffset` objects returning an `object` dtype instead of `datetime64[ns]` dtype ([GH21610](https://github.com/pandas-dev/pandas/issues/21610), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DataFrame` with `datetime64[ns]` dtype comparing against `NaT` incorrectly ([GH22242](https://github.com/pandas-dev/pandas/issues/22242), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DataFrame` with `datetime64[ns]` dtype subtracting `Timestamp`-like object incorrectly returned `datetime64[ns]` dtype instead of `timedelta64[ns]` dtype ([GH8554](https://github.com/pandas-dev/pandas/issues/8554), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DataFrame` with `datetime64[ns]` dtype subtracting `np.datetime64` object with non-nanosecond unit failing to convert to nanoseconds ([GH18874](https://github.com/pandas-dev/pandas/issues/18874), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DataFrame` comparisons against `Timestamp`-like objects failing to raise `TypeError` for inequality checks with mismatched types ([GH8932](https://github.com/pandas-dev/pandas/issues/8932), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DataFrame` with mixed dtypes including `datetime64[ns]` incorrectly raising `TypeError` on equality comparisons ([GH13128](https://github.com/pandas-dev/pandas/issues/13128), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DataFrame.values` returning a `DatetimeIndex` for a single-column `DataFrame` with tz-aware datetime values. Now a 2-D [`numpy.ndarray`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) of `Timestamp` objects is returned ([GH24024](https://github.com/pandas-dev/pandas/issues/24024))  
- Bug in `DataFrame.eq()` comparison against `NaT` incorrectly returning `True` or `NaN` ([GH15697](https://github.com/pandas-dev/pandas/issues/15697), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DatetimeIndex` subtraction that incorrectly failed to raise `OverflowError` ([GH22492](https://github.com/pandas-dev/pandas/issues/22492), [GH22508](https://github.com/pandas-dev/pandas/issues/22508))  
- Bug in `DatetimeIndex` incorrectly allowing indexing with `Timedelta` object ([GH20464](https://github.com/pandas-dev/pandas/issues/20464))  
- Bug in `DatetimeIndex` where frequency was being set if original frequency was `None` ([GH22150](https://github.com/pandas-dev/pandas/issues/22150))  
- Bug in rounding methods of `DatetimeIndex` (`round()`, `ceil()`, `floor()`) and `Timestamp` (`round()`, `ceil()`, `floor()`) could give rise to loss of precision ([GH22591](https://github.com/pandas-dev/pandas/issues/22591))  
- Bug in `to_datetime()` with an `Index` argument that would drop the `name` from the result ([GH21697](https://github.com/pandas-dev/pandas/issues/21697))  
- Bug in `PeriodIndex` where adding or subtracting a `timedelta` or `Tick` object produced incorrect results ([GH22988](https://github.com/pandas-dev/pandas/issues/22988))  
- Bug in the `Series` repr with period-dtype data missing a space before the data ([GH23601](https://github.com/pandas-dev/pandas/issues/23601))  
- Bug in `date_range()` when decrementing a start date to a past end date by a negative frequency ([GH23270](https://github.com/pandas-dev/pandas/issues/23270))  
- Bug in `Series.min()` which would return `NaN` instead of `NaT` when called on a series of `NaT` ([GH23282](https://github.com/pandas-dev/pandas/issues/23282))  
- Bug in `Series.combine_first()` not properly aligning categoricals, so that missing values in `self` where not filled by valid values from `other` ([GH24147](https://github.com/pandas-dev/pandas/issues/24147))  
- Bug in `DataFrame.combine()` with datetimelike values raising a TypeError ([GH23079](https://github.com/pandas-dev/pandas/issues/23079))  
- Bug in `date_range()` with frequency of `Day` or higher where dates sufficiently far in the future could wrap around to the past instead of raising `OutOfBoundsDatetime` ([GH14187](https://github.com/pandas-dev/pandas/issues/14187))  
- Bug in `period_range()` ignoring the frequency of `start` and `end` when those are provided as `Period` objects ([GH20535](https://github.com/pandas-dev/pandas/issues/20535)).  
- Bug in `PeriodIndex` with attribute `freq.n` greater than 1 where adding a `DateOffset` object would return incorrect results ([GH23215](https://github.com/pandas-dev/pandas/issues/23215))  
- Bug in `Series` that interpreted string indices as lists of characters when setting datetimelike values ([GH23451](https://github.com/pandas-dev/pandas/issues/23451))  
- Bug in `DataFrame` when creating a new column from an ndarray of `Timestamp` objects with timezones creating an object-dtype column, rather than datetime with timezone ([GH23932](https://github.com/pandas-dev/pandas/issues/23932))  
- Bug in `Timestamp` constructor which would drop the frequency of an input `Timestamp` ([GH22311](https://github.com/pandas-dev/pandas/issues/22311))  
- Bug in `DatetimeIndex` where calling `np.array(dtindex, dtype=object)` would incorrectly return an array of `long` objects ([GH23524](https://github.com/pandas-dev/pandas/issues/23524))  
- Bug in `Index` where passing a timezone-aware `DatetimeIndex` and dtype=object would incorrectly raise a `ValueError` ([GH23524](https://github.com/pandas-dev/pandas/issues/23524))  
- Bug in `Index` where calling `np.array(dtindex, dtype=object)` on a timezone-naive `DatetimeIndex` would return an array of `datetime` objects instead of `Timestamp` objects, potentially losing nanosecond portions of the timestamps ([GH23524](https://github.com/pandas-dev/pandas/issues/23524))  
- Bug in `Categorical.__setitem__` not allowing setting with another `Categorical` when both are unordered and have the same categories, but in a different order ([GH24142](https://github.com/pandas-dev/pandas/issues/24142))  
- Bug in `date_range()` where using dates with millisecond resolution or higher could return incorrect values or the wrong number of values in the index ([GH24110](https://github.com/pandas-dev/pandas/issues/24110))  
- Bug in `DatetimeIndex` where constructing a `DatetimeIndex` from a `Categorical` or `CategoricalIndex` would incorrectly drop timezone information ([GH18664](https://github.com/pandas-dev/pandas/issues/18664))  
- Bug in `DatetimeIndex` and `TimedeltaIndex` where indexing with `Ellipsis` would incorrectly lose the index’s `freq` attribute ([GH21282](https://github.com/pandas-dev/pandas/issues/21282))  
- Clarified error message produced when passing an incorrect `freq` argument to `DatetimeIndex` with `NaT` as the first entry in the passed data ([GH11587](https://github.com/pandas-dev/pandas/issues/11587))  
- Bug in `to_datetime()` where `box` and `utc` arguments were ignored when passing a `DataFrame` or `dict` of unit mappings ([GH23760](https://github.com/pandas-dev/pandas/issues/23760))  
- Bug in `Series.dt` where the cache would not update properly after an in-place operation ([GH24408](https://github.com/pandas-dev/pandas/issues/24408))  
- Bug in `PeriodIndex` where comparisons against an array-like object with length 1 failed to raise `ValueError` ([GH23078](https://github.com/pandas-dev/pandas/issues/23078))  
- Bug in `DatetimeIndex.astype()`, `PeriodIndex.astype()` and `TimedeltaIndex.astype()` ignoring the sign of the `dtype` for unsigned integer dtypes ([GH24405](https://github.com/pandas-dev/pandas/issues/24405)).  
- Fixed bug in `Series.max()` with `datetime64[ns]`-dtype failing to return `NaT` when nulls are present and `skipna=False` is passed ([GH24265](https://github.com/pandas-dev/pandas/issues/24265))  
- Bug in `to_datetime()` where arrays of `datetime` objects containing both timezone-aware and timezone-naive `datetimes` would fail to raise `ValueError` ([GH24569](https://github.com/pandas-dev/pandas/issues/24569))  
- Bug in `to_datetime()` with invalid datetime format doesn’t coerce input to `NaT` even if `errors='coerce'` ([GH24763](https://github.com/pandas-dev/pandas/issues/24763))  

### Timedelta

- Bug in `DataFrame` with `timedelta64[ns]` dtype division by `Timedelta`-like scalar incorrectly returning `timedelta64[ns]` dtype instead of `float64` dtype ([GH20088](https://github.com/pandas-dev/pandas/issues/20088), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in adding a `Index` with object dtype to a `Series` with `timedelta64[ns]` dtype incorrectly raising ([GH22390](https://github.com/pandas-dev/pandas/issues/22390))  
- Bug in multiplying a `Series` with numeric dtype against a `timedelta` object ([GH22390](https://github.com/pandas-dev/pandas/issues/22390))  
- Bug in `Series` with numeric dtype when adding or subtracting an an array or `Series` with `timedelta64` dtype ([GH22390](https://github.com/pandas-dev/pandas/issues/22390))  
- Bug in `Index` with numeric dtype when multiplying or dividing an array with dtype `timedelta64` ([GH22390](https://github.com/pandas-dev/pandas/issues/22390))  
- Bug in `TimedeltaIndex` incorrectly allowing indexing with `Timestamp` object ([GH20464](https://github.com/pandas-dev/pandas/issues/20464))  
- Fixed bug where subtracting `Timedelta` from an object-dtyped array would raise `TypeError` ([GH21980](https://github.com/pandas-dev/pandas/issues/21980))  
- Fixed bug in adding a `DataFrame` with all-timedelta64[ns] dtypes to a `DataFrame` with all-integer dtypes returning incorrect results instead of raising `TypeError` ([GH22696](https://github.com/pandas-dev/pandas/issues/22696))  
- Bug in `TimedeltaIndex` where adding a timezone-aware datetime scalar incorrectly returned a timezone-naive `DatetimeIndex` ([GH23215](https://github.com/pandas-dev/pandas/issues/23215))  
- Bug in `TimedeltaIndex` where adding `np.timedelta64('NaT')` incorrectly returned an all-`NaT` `DatetimeIndex` instead of an all-`NaT` `TimedeltaIndex` ([GH23215](https://github.com/pandas-dev/pandas/issues/23215))  
- Bug in `Timedelta` and `to_timedelta()` have inconsistencies in supported unit string ([GH21762](https://github.com/pandas-dev/pandas/issues/21762))  
- Bug in `TimedeltaIndex` division where dividing by another `TimedeltaIndex` raised `TypeError` instead of returning a `Float64Index` ([GH23829](https://github.com/pandas-dev/pandas/issues/23829), [GH22631](https://github.com/pandas-dev/pandas/issues/22631))  
- Bug in `TimedeltaIndex` comparison operations where comparing against non-`Timedelta`-like objects would raise `TypeError` instead of returning all-`False` for `__eq__` and all-`True` for `__ne__` ([GH24056](https://github.com/pandas-dev/pandas/issues/24056))  
- Bug in `Timedelta` comparisons when comparing with a `Tick` object incorrectly raising `TypeError` ([GH24710](https://github.com/pandas-dev/pandas/issues/24710))  

### Timezones

- Bug in `Index.shift()` where an `AssertionError` would raise when shifting across DST ([GH8616](https://github.com/pandas-dev/pandas/issues/8616))  
- Bug in `Timestamp` constructor where passing an invalid timezone offset designator (`Z`) would not raise a `ValueError` ([GH8910](https://github.com/pandas-dev/pandas/issues/8910))  
- Bug in `Timestamp.replace()` where replacing at a DST boundary would retain an incorrect offset ([GH7825](https://github.com/pandas-dev/pandas/issues/7825))  
- Bug in `Series.replace()` with `datetime64[ns, tz]` data when replacing `NaT` ([GH11792](https://github.com/pandas-dev/pandas/issues/11792))  
- Bug in `Timestamp` when passing different string date formats with a timezone offset would produce different timezone offsets ([GH12064](https://github.com/pandas-dev/pandas/issues/12064))  
- Bug when comparing a tz-naive `Timestamp` to a tz-aware `DatetimeIndex` which would coerce the `DatetimeIndex` to tz-naive ([GH12601](https://github.com/pandas-dev/pandas/issues/12601))  
- Bug in `Series.truncate()` with a tz-aware `DatetimeIndex` which would cause a core dump ([GH9243](https://github.com/pandas-dev/pandas/issues/9243))  
- Bug in `Series` constructor which would coerce tz-aware and tz-naive `Timestamp` to tz-aware ([GH13051](https://github.com/pandas-dev/pandas/issues/13051))  
- Bug in `Index` with `datetime64[ns, tz]` dtype that did not localize integer data correctly ([GH20964](https://github.com/pandas-dev/pandas/issues/20964))  
- Bug in `DatetimeIndex` where constructing with an integer and tz would not localize correctly ([GH12619](https://github.com/pandas-dev/pandas/issues/12619))  
- Fixed bug where `DataFrame.describe()` and `Series.describe()` on tz-aware datetimes did not show first and last result ([GH21328](https://github.com/pandas-dev/pandas/issues/21328))  
- Bug in `DatetimeIndex` comparisons failing to raise `TypeError` when comparing timezone-aware `DatetimeIndex` against `np.datetime64` ([GH22074](https://github.com/pandas-dev/pandas/issues/22074))  
- Bug in `DataFrame` assignment with a timezone-aware scalar ([GH19843](https://github.com/pandas-dev/pandas/issues/19843))  
- Bug in `DataFrame.asof()` that raised a `TypeError` when attempting to compare tz-naive and tz-aware timestamps ([GH21194](https://github.com/pandas-dev/pandas/issues/21194))  
- Bug when constructing a `DatetimeIndex` with `Timestamp` constructed with the `replace` method across DST ([GH18785](https://github.com/pandas-dev/pandas/issues/18785))  
- Bug when setting a new value with `DataFrame.loc()` with a `DatetimeIndex` with a DST transition ([GH18308](https://github.com/pandas-dev/pandas/issues/18308), [GH20724](https://github.com/pandas-dev/pandas/issues/20724))  
- Bug in `Index.unique()` that did not re-localize tz-aware dates correctly ([GH21737](https://github.com/pandas-dev/pandas/issues/21737))  
- Bug when indexing a `Series` with a DST transition ([GH21846](https://github.com/pandas-dev/pandas/issues/21846))  
- Bug in `DataFrame.resample()` and `Series.resample()` where an `AmbiguousTimeError` or `NonExistentTimeError` would raise if a timezone aware timeseries ended on a DST transition ([GH19375](https://github.com/pandas-dev/pandas/issues/19375), [GH10117](https://github.com/pandas-dev/pandas/issues/10117))  
- Bug in `DataFrame.drop()` and `Series.drop()` when specifying a tz-aware Timestamp key to drop from a `DatetimeIndex` with a DST transition ([GH21761](https://github.com/pandas-dev/pandas/issues/21761))  
- Bug in `DatetimeIndex` constructor where `NaT` and `dateutil.tz.tzlocal` would raise an `OutOfBoundsDatetime` error ([GH23807](https://github.com/pandas-dev/pandas/issues/23807))  
- Bug in `DatetimeIndex.tz_localize()` and `Timestamp.tz_localize()` with `dateutil.tz.tzlocal` near a DST transition that would return an incorrectly localized datetime ([GH23807](https://github.com/pandas-dev/pandas/issues/23807))  
- Bug in `Timestamp` constructor where a `dateutil.tz.tzutc` timezone passed with a `datetime.datetime` argument would be converted to a `pytz.UTC` timezone ([GH23807](https://github.com/pandas-dev/pandas/issues/23807))  
- Bug in `to_datetime()` where `utc=True` was not respected when specifying a `unit` and `errors='ignore'` ([GH23758](https://github.com/pandas-dev/pandas/issues/23758))  
- Bug in `to_datetime()` where `utc=True` was not respected when passing a `Timestamp` ([GH24415](https://github.com/pandas-dev/pandas/issues/24415))  
- Bug in `DataFrame.any()` returns wrong value when `axis=1` and the data is of datetimelike type ([GH23070](https://github.com/pandas-dev/pandas/issues/23070))  
- Bug in `DatetimeIndex.to_period()` where a timezone aware index was converted to UTC first before creating `PeriodIndex` ([GH22905](https://github.com/pandas-dev/pandas/issues/22905))  
- Bug in `DataFrame.tz_localize()`, `DataFrame.tz_convert()`, `Series.tz_localize()`, and `Series.tz_convert()` where `copy=False` would mutate the original argument inplace ([GH6326](https://github.com/pandas-dev/pandas/issues/6326))  
- Bug in `DataFrame.max()` and `DataFrame.min()` with `axis=1` where a `Series` with `NaN` would be returned when all columns contained the same timezone ([GH10390](https://github.com/pandas-dev/pandas/issues/10390))  

### Offsets

- Bug in `FY5253` where date offsets could incorrectly raise an `AssertionError` in arithmetic operations ([GH14774](https://github.com/pandas-dev/pandas/issues/14774))  
- Bug in `DateOffset` where keyword arguments `week` and `milliseconds` were accepted and ignored.  Passing these will now raise `ValueError` ([GH19398](https://github.com/pandas-dev/pandas/issues/19398))  
- Bug in adding `DateOffset` with `DataFrame` or `PeriodIndex` incorrectly raising `TypeError` ([GH23215](https://github.com/pandas-dev/pandas/issues/23215))  
- Bug in comparing `DateOffset` objects with non-DateOffset objects, particularly strings, raising `ValueError` instead of returning `False` for equality checks and `True` for not-equal checks ([GH23524](https://github.com/pandas-dev/pandas/issues/23524))  

### Numeric

- Bug in `Series` `__rmatmul__` doesn’t support matrix vector multiplication ([GH21530](https://github.com/pandas-dev/pandas/issues/21530))  
- Bug in `factorize()` fails with read-only array ([GH12813](https://github.com/pandas-dev/pandas/issues/12813))  
- Fixed bug in `unique()` handled signed zeros inconsistently: for some inputs 0.0 and -0.0 were treated as equal and for some inputs as different. Now they are treated as equal for all inputs ([GH21866](https://github.com/pandas-dev/pandas/issues/21866))  
- Bug in `DataFrame.agg()`, `DataFrame.transform()` and `DataFrame.apply()` where,
  when supplied with a list of functions and `axis=1` (e.g. `df.apply(['sum', 'mean'], axis=1)`),
  a `TypeError` was wrongly raised. For all three methods such calculation are now done correctly. ([GH16679](https://github.com/pandas-dev/pandas/issues/16679)).  
- Bug in `Series` comparison against datetime-like scalars and arrays ([GH22074](https://github.com/pandas-dev/pandas/issues/22074))  
- Bug in `DataFrame` multiplication between boolean dtype and integer returning `object` dtype instead of integer dtype ([GH22047](https://github.com/pandas-dev/pandas/issues/22047), [GH22163](https://github.com/pandas-dev/pandas/issues/22163))  
- Bug in `DataFrame.apply()` where, when supplied with a string argument and additional positional or keyword arguments (e.g. `df.apply('sum', min_count=1)`), a `TypeError` was wrongly raised ([GH22376](https://github.com/pandas-dev/pandas/issues/22376))  
- Bug in `DataFrame.astype()` to extension dtype may raise `AttributeError` ([GH22578](https://github.com/pandas-dev/pandas/issues/22578))  
- Bug in `DataFrame` with `timedelta64[ns]` dtype arithmetic operations with `ndarray` with integer dtype incorrectly treating the narray as `timedelta64[ns]` dtype ([GH23114](https://github.com/pandas-dev/pandas/issues/23114))  
- Bug in `Series.rpow()` with object dtype `NaN` for `1 ** NA` instead of `1` ([GH22922](https://github.com/pandas-dev/pandas/issues/22922)).  
- `Series.agg()` can now handle numpy NaN-aware methods like [`numpy.nansum()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.nansum.html#numpy.nansum) ([GH19629](https://github.com/pandas-dev/pandas/issues/19629))  
- Bug in `Series.rank()` and `DataFrame.rank()` when `pct=True` and more than 224 rows are present resulted in percentages greater than 1.0 ([GH18271](https://github.com/pandas-dev/pandas/issues/18271))  
- Calls such as `DataFrame.round()` with a non-unique `CategoricalIndex()` now return expected data. Previously, data would be improperly duplicated ([GH21809](https://github.com/pandas-dev/pandas/issues/21809)).  
- Added `log10`, floor and ceil to the list of supported functions in `DataFrame.eval()` ([GH24139](https://github.com/pandas-dev/pandas/issues/24139), [GH24353](https://github.com/pandas-dev/pandas/issues/24353))  
- Logical operations `&, |, ^` between `Series` and `Index` will no longer raise `ValueError` ([GH22092](https://github.com/pandas-dev/pandas/issues/22092))  
- Checking PEP 3141 numbers in `is_scalar()` function returns `True` ([GH22903](https://github.com/pandas-dev/pandas/issues/22903))  
- Reduction methods like `Series.sum()` now accept the default value of `keepdims=False` when called from a NumPy ufunc, rather than raising a `TypeError`. Full support for `keepdims` has not been implemented ([GH24356](https://github.com/pandas-dev/pandas/issues/24356)).  

### Conversion

- Bug in `DataFrame.combine_first()` in which column types were unexpectedly converted to float ([GH20699](https://github.com/pandas-dev/pandas/issues/20699))  
- Bug in `DataFrame.clip()` in which column types are not preserved and casted to float ([GH24162](https://github.com/pandas-dev/pandas/issues/24162))  
- Bug in `DataFrame.clip()` when order of columns of dataframes doesn’t match, result observed is wrong in numeric values ([GH20911](https://github.com/pandas-dev/pandas/issues/20911))  
- Bug in `DataFrame.astype()` where converting to an extension dtype when duplicate column names are present causes a `RecursionError` ([GH24704](https://github.com/pandas-dev/pandas/issues/24704))  

### Strings

- Bug in `Index.str.partition()` was not nan-safe ([GH23558](https://github.com/pandas-dev/pandas/issues/23558)).  
- Bug in `Index.str.split()` was not nan-safe ([GH23677](https://github.com/pandas-dev/pandas/issues/23677)).  
- Bug `Series.str.contains()` not respecting the `na` argument for a `Categorical` dtype `Series` ([GH22158](https://github.com/pandas-dev/pandas/issues/22158))  
- Bug in `Index.str.cat()` when the result contained only `NaN` ([GH24044](https://github.com/pandas-dev/pandas/issues/24044))  

### Interval

- Bug in the `IntervalIndex` constructor where the `closed` parameter did not always override the inferred `closed` ([GH19370](https://github.com/pandas-dev/pandas/issues/19370))  
- Bug in the `IntervalIndex` repr where a trailing comma was missing after the list of intervals ([GH20611](https://github.com/pandas-dev/pandas/issues/20611))  
- Bug in `Interval` where scalar arithmetic operations did not retain the `closed` value ([GH22313](https://github.com/pandas-dev/pandas/issues/22313))  
- Bug in `IntervalIndex` where indexing with datetime-like values raised a `KeyError` ([GH20636](https://github.com/pandas-dev/pandas/issues/20636))  
- Bug in `IntervalTree` where data containing `NaN` triggered a warning and resulted in incorrect indexing queries with `IntervalIndex` ([GH23352](https://github.com/pandas-dev/pandas/issues/23352))  

### Indexing

- Bug in `DataFrame.ne()` fails if columns contain column name “dtype” ([GH22383](https://github.com/pandas-dev/pandas/issues/22383))  
- The traceback from a `KeyError` when asking `.loc` for a single missing label is now shorter and more clear ([GH21557](https://github.com/pandas-dev/pandas/issues/21557))  
- `PeriodIndex` now emits a `KeyError` when a malformed string is looked up, which is consistent with the behavior of `DatetimeIndex` ([GH22803](https://github.com/pandas-dev/pandas/issues/22803))  
- When `.ix` is asked for a missing integer label in a `MultiIndex` with a first level of integer type, it now raises a `KeyError`, consistently with the case of a flat `Int64Index`, rather than falling back to positional indexing ([GH21593](https://github.com/pandas-dev/pandas/issues/21593))  
- Bug in `Index.reindex()` when reindexing a tz-naive and tz-aware `DatetimeIndex` ([GH8306](https://github.com/pandas-dev/pandas/issues/8306))  
- Bug in `Series.reindex()` when reindexing an empty series with a `datetime64[ns, tz]` dtype ([GH20869](https://github.com/pandas-dev/pandas/issues/20869))  
- Bug in `DataFrame` when setting values with `.loc` and a timezone aware `DatetimeIndex` ([GH11365](https://github.com/pandas-dev/pandas/issues/11365))  
- `DataFrame.__getitem__` now accepts dictionaries and dictionary keys as list-likes of labels, consistently with `Series.__getitem__` ([GH21294](https://github.com/pandas-dev/pandas/issues/21294))  
- Fixed `DataFrame[np.nan]` when columns are non-unique ([GH21428](https://github.com/pandas-dev/pandas/issues/21428))  
- Bug when indexing `DatetimeIndex` with nanosecond resolution dates and timezones ([GH11679](https://github.com/pandas-dev/pandas/issues/11679))  
- Bug where indexing with a Numpy array containing negative values would mutate the indexer ([GH21867](https://github.com/pandas-dev/pandas/issues/21867))  
- Bug where mixed indexes wouldn’t allow integers for `.at` ([GH19860](https://github.com/pandas-dev/pandas/issues/19860))  
- `Float64Index.get_loc` now raises `KeyError` when boolean key passed. ([GH19087](https://github.com/pandas-dev/pandas/issues/19087))  
- Bug in `DataFrame.loc()` when indexing with an `IntervalIndex` ([GH19977](https://github.com/pandas-dev/pandas/issues/19977))  
- `Index` no longer mangles `None`, `NaN` and `NaT`, i.e. they are treated as three different keys. However, for numeric Index all three are still coerced to a `NaN` ([GH22332](https://github.com/pandas-dev/pandas/issues/22332))  
- Bug in `scalar in Index` if scalar is a float while the `Index` is of integer dtype ([GH22085](https://github.com/pandas-dev/pandas/issues/22085))  
- Bug in `MultiIndex.set_levels()` when levels value is not subscriptable ([GH23273](https://github.com/pandas-dev/pandas/issues/23273))  
- Bug where setting a timedelta column by `Index` causes it to be casted to double, and therefore lose precision ([GH23511](https://github.com/pandas-dev/pandas/issues/23511))  
- Bug in `Index.union()` and `Index.intersection()` where name of the `Index` of the result was not computed correctly for certain cases ([GH9943](https://github.com/pandas-dev/pandas/issues/9943), [GH9862](https://github.com/pandas-dev/pandas/issues/9862))  
- Bug in `Index` slicing with boolean `Index` may raise `TypeError` ([GH22533](https://github.com/pandas-dev/pandas/issues/22533))  
- Bug in `PeriodArray.__setitem__` when accepting slice and list-like value ([GH23978](https://github.com/pandas-dev/pandas/issues/23978))  
- Bug in `DatetimeIndex`, `TimedeltaIndex` where indexing with `Ellipsis` would lose their `freq` attribute ([GH21282](https://github.com/pandas-dev/pandas/issues/21282))  
- Bug in `iat` where using it to assign an incompatible value would create a new column ([GH23236](https://github.com/pandas-dev/pandas/issues/23236))  

### Missing

- Bug in `DataFrame.fillna()` where a `ValueError` would raise when one column contained a `datetime64[ns, tz]` dtype ([GH15522](https://github.com/pandas-dev/pandas/issues/15522))  
- Bug in `Series.hasnans()` that could be incorrectly cached and return incorrect answers if null elements are introduced after an initial call ([GH19700](https://github.com/pandas-dev/pandas/issues/19700))  
- `Series.isin()` now treats all NaN-floats as equal also for `np.object`-dtype. This behavior is consistent with the behavior for float64 ([GH22119](https://github.com/pandas-dev/pandas/issues/22119))  
- `unique()` no longer mangles NaN-floats and the `NaT`-object for `np.object`-dtype, i.e. `NaT` is no longer coerced to a NaN-value and is treated as a different entity. ([GH22295](https://github.com/pandas-dev/pandas/issues/22295))  
- `DataFrame` and `Series` now properly handle numpy masked arrays with hardened masks. Previously, constructing a DataFrame or Series from a masked array with a hard mask would create a pandas object containing the underlying value, rather than the expected NaN. ([GH24574](https://github.com/pandas-dev/pandas/issues/24574))  
- Bug in `DataFrame` constructor where `dtype` argument was not honored when handling numpy masked record arrays. ([GH24874](https://github.com/pandas-dev/pandas/issues/24874))  

### MultiIndex

- Bug in `io.formats.style.Styler.applymap()` where `subset=` with `MultiIndex` slice would reduce to `Series` ([GH19861](https://github.com/pandas-dev/pandas/issues/19861))  
- Removed compatibility for `MultiIndex` pickles prior to version 0.8.0; compatibility with `MultiIndex` pickles from version 0.13 forward is maintained ([GH21654](https://github.com/pandas-dev/pandas/issues/21654))  
- `MultiIndex.get_loc_level()` (and as a consequence, `.loc` on a `Series` or `DataFrame` with a `MultiIndex` index) will now raise a `KeyError`, rather than returning an empty `slice`, if asked a label which is present in the `levels` but is unused ([GH22221](https://github.com/pandas-dev/pandas/issues/22221))  
- `MultiIndex` has gained the `MultiIndex.from_frame()`, it allows constructing a `MultiIndex` object from a `DataFrame` ([GH22420](https://github.com/pandas-dev/pandas/issues/22420))  
- Fix `TypeError` in Python 3 when creating `MultiIndex` in which some levels have mixed types, e.g. when some labels are tuples ([GH15457](https://github.com/pandas-dev/pandas/issues/15457))  

### I/O

- Bug in `read_csv()` in which a column specified with `CategoricalDtype` of boolean categories was not being correctly coerced from string values to booleans ([GH20498](https://github.com/pandas-dev/pandas/issues/20498))  
- Bug in `read_csv()` in which unicode column names were not being properly recognized with Python 2.x ([GH13253](https://github.com/pandas-dev/pandas/issues/13253))  
- Bug in `DataFrame.to_sql()` when writing timezone aware data (`datetime64[ns, tz]` dtype) would raise a `TypeError` ([GH9086](https://github.com/pandas-dev/pandas/issues/9086))  
- Bug in `DataFrame.to_sql()` where a naive `DatetimeIndex` would be written as `TIMESTAMP WITH TIMEZONE` type in supported databases, e.g. PostgreSQL ([GH23510](https://github.com/pandas-dev/pandas/issues/23510))  
- Bug in `read_excel()` when `parse_cols` is specified with an empty dataset ([GH9208](https://github.com/pandas-dev/pandas/issues/9208))  
- `read_html()` no longer ignores all-whitespace `<tr>` within `<thead>` when considering the `skiprows` and `header` arguments. Previously, users had to decrease their `header` and `skiprows` values on such tables to work around the issue. ([GH21641](https://github.com/pandas-dev/pandas/issues/21641))  
- `read_excel()` will correctly show the deprecation warning for previously deprecated `sheetname` ([GH17994](https://github.com/pandas-dev/pandas/issues/17994))  
- `read_csv()` and `read_table()` will throw `UnicodeError` and not coredump on badly encoded strings ([GH22748](https://github.com/pandas-dev/pandas/issues/22748))  
- `read_csv()` will correctly parse timezone-aware datetimes ([GH22256](https://github.com/pandas-dev/pandas/issues/22256))  
- Bug in `read_csv()` in which memory management was prematurely optimized for the C engine when the data was being read in chunks ([GH23509](https://github.com/pandas-dev/pandas/issues/23509))  
- Bug in `read_csv()` in unnamed columns were being improperly identified when extracting a multi-index ([GH23687](https://github.com/pandas-dev/pandas/issues/23687))  
- `read_sas()` will parse numbers in sas7bdat-files that have width less than 8 bytes correctly. ([GH21616](https://github.com/pandas-dev/pandas/issues/21616))  
- `read_sas()` will correctly parse sas7bdat files with many columns ([GH22628](https://github.com/pandas-dev/pandas/issues/22628))  
- `read_sas()` will correctly parse sas7bdat files with data page types having also bit 7 set (so page type is 128 + 256 = 384) ([GH16615](https://github.com/pandas-dev/pandas/issues/16615))  
- Bug in `read_sas()` in which an incorrect error was raised on an invalid file format. ([GH24548](https://github.com/pandas-dev/pandas/issues/24548))  
- Bug in `detect_client_encoding()` where potential `IOError` goes unhandled when importing in a mod_wsgi process due to restricted access to stdout. ([GH21552](https://github.com/pandas-dev/pandas/issues/21552))  
- Bug in `DataFrame.to_html()` with `index=False` misses truncation indicators (…) on truncated DataFrame ([GH15019](https://github.com/pandas-dev/pandas/issues/15019), [GH22783](https://github.com/pandas-dev/pandas/issues/22783))  
- Bug in `DataFrame.to_html()` with `index=False` when both columns and row index are `MultiIndex` ([GH22579](https://github.com/pandas-dev/pandas/issues/22579))  
- Bug in `DataFrame.to_html()` with `index_names=False` displaying index name ([GH22747](https://github.com/pandas-dev/pandas/issues/22747))  
- Bug in `DataFrame.to_html()` with `header=False` not displaying row index names ([GH23788](https://github.com/pandas-dev/pandas/issues/23788))  
- Bug in `DataFrame.to_html()` with `sparsify=False` that caused it to raise `TypeError` ([GH22887](https://github.com/pandas-dev/pandas/issues/22887))  
- Bug in `DataFrame.to_string()` that broke column alignment when `index=False` and width of first column’s values is greater than the width of first column’s header ([GH16839](https://github.com/pandas-dev/pandas/issues/16839), [GH13032](https://github.com/pandas-dev/pandas/issues/13032))  
- Bug in `DataFrame.to_string()` that caused representations of `DataFrame` to not take up the whole window ([GH22984](https://github.com/pandas-dev/pandas/issues/22984))  
- Bug in `DataFrame.to_csv()` where a single level MultiIndex incorrectly wrote a tuple. Now just the value of the index is written ([GH19589](https://github.com/pandas-dev/pandas/issues/19589)).  
- `HDFStore` will raise `ValueError` when the `format` kwarg is passed to the constructor ([GH13291](https://github.com/pandas-dev/pandas/issues/13291))  
- Bug in `HDFStore.append()` when appending a `DataFrame` with an empty string column and `min_itemsize` < 8 ([GH12242](https://github.com/pandas-dev/pandas/issues/12242))  
- Bug in `read_csv()` in which memory leaks occurred in the C engine when parsing `NaN` values due to insufficient cleanup on completion or error ([GH21353](https://github.com/pandas-dev/pandas/issues/21353))  
- Bug in `read_csv()` in which incorrect error messages were being raised when `skipfooter` was passed in along with `nrows`, `iterator`, or `chunksize` ([GH23711](https://github.com/pandas-dev/pandas/issues/23711))  
- Bug in `read_csv()` in which `MultiIndex` index names were being improperly handled in the cases when they were not provided ([GH23484](https://github.com/pandas-dev/pandas/issues/23484))  
- Bug in `read_csv()` in which unnecessary warnings were being raised when the dialect’s values conflicted with the default arguments ([GH23761](https://github.com/pandas-dev/pandas/issues/23761))  
- Bug in `read_html()` in which the error message was not displaying the valid flavors when an invalid one was provided ([GH23549](https://github.com/pandas-dev/pandas/issues/23549))  
- Bug in `read_excel()` in which extraneous header names were extracted, even though none were specified ([GH11733](https://github.com/pandas-dev/pandas/issues/11733))  
- Bug in `read_excel()` in which column names were not being properly converted to string sometimes in Python 2.x ([GH23874](https://github.com/pandas-dev/pandas/issues/23874))  
- Bug in `read_excel()` in which `index_col=None` was not being respected and parsing index columns anyway ([GH18792](https://github.com/pandas-dev/pandas/issues/18792), [GH20480](https://github.com/pandas-dev/pandas/issues/20480))  
- Bug in `read_excel()` in which `usecols` was not being validated for proper column names when passed in as a string ([GH20480](https://github.com/pandas-dev/pandas/issues/20480))  
- Bug in `DataFrame.to_dict()` when the resulting dict contains non-Python scalars in the case of numeric data ([GH23753](https://github.com/pandas-dev/pandas/issues/23753))  
- `DataFrame.to_string()`, `DataFrame.to_html()`, `DataFrame.to_latex()` will correctly format output when a string is passed as the `float_format` argument ([GH21625](https://github.com/pandas-dev/pandas/issues/21625), [GH22270](https://github.com/pandas-dev/pandas/issues/22270))  
- Bug in `read_csv()` that caused it to raise `OverflowError` when trying to use ‘inf’ as `na_value` with integer index column ([GH17128](https://github.com/pandas-dev/pandas/issues/17128))  
- Bug in `read_csv()` that caused the C engine on Python 3.6+ on Windows to improperly read CSV filenames with accented or special characters ([GH15086](https://github.com/pandas-dev/pandas/issues/15086))  
- Bug in `read_fwf()` in which the compression type of a file was not being properly inferred ([GH22199](https://github.com/pandas-dev/pandas/issues/22199))  
- Bug in `pandas.io.json.json_normalize()` that caused it to raise `TypeError` when two consecutive elements of `record_path` are dicts ([GH22706](https://github.com/pandas-dev/pandas/issues/22706))  
- Bug in `DataFrame.to_stata()`, `pandas.io.stata.StataWriter` and `pandas.io.stata.StataWriter117` where a exception would leave a partially written and invalid dta file ([GH23573](https://github.com/pandas-dev/pandas/issues/23573))  
- Bug in `DataFrame.to_stata()` and `pandas.io.stata.StataWriter117` that produced invalid files when using strLs with non-ASCII characters ([GH23573](https://github.com/pandas-dev/pandas/issues/23573))  
- Bug in `HDFStore` that caused it to raise `ValueError` when reading a Dataframe in Python 3 from fixed format written in Python 2 ([GH24510](https://github.com/pandas-dev/pandas/issues/24510))  
- Bug in `DataFrame.to_string()` and more generally in the floating `repr` formatter. Zeros were not trimmed if `inf` was present in a columns while it was the case with NA values. Zeros are now trimmed as in the presence of NA ([GH24861](https://github.com/pandas-dev/pandas/issues/24861)).  
- Bug in the `repr` when truncating the number of columns and having a wide last column ([GH24849](https://github.com/pandas-dev/pandas/issues/24849)).  

### Plotting

- Bug in `DataFrame.plot.scatter()` and `DataFrame.plot.hexbin()` caused x-axis label and ticklabels to disappear when colorbar was on in IPython inline backend ([GH10611](https://github.com/pandas-dev/pandas/issues/10611), [GH10678](https://github.com/pandas-dev/pandas/issues/10678), and [GH20455](https://github.com/pandas-dev/pandas/issues/20455))  
- Bug in plotting a Series with datetimes using `matplotlib.axes.Axes.scatter()` ([GH22039](https://github.com/pandas-dev/pandas/issues/22039))  
- Bug in `DataFrame.plot.bar()` caused bars to use multiple colors instead of a single one ([GH20585](https://github.com/pandas-dev/pandas/issues/20585))  
- Bug in validating color parameter caused extra color to be appended to the given color array. This happened to multiple plotting functions using matplotlib. ([GH20726](https://github.com/pandas-dev/pandas/issues/20726))  

### Groupby/resample/rolling

- Bug in `pandas.core.window.Rolling.min()` and `pandas.core.window.Rolling.max()` with `closed='left'`, a datetime-like index and only one entry in the series leading to segfault ([GH24718](https://github.com/pandas-dev/pandas/issues/24718))  
- Bug in `pandas.core.groupby.GroupBy.first()` and `pandas.core.groupby.GroupBy.last()` with `as_index=False` leading to the loss of timezone information ([GH15884](https://github.com/pandas-dev/pandas/issues/15884))  
- Bug in `DateFrame.resample()` when downsampling across a DST boundary ([GH8531](https://github.com/pandas-dev/pandas/issues/8531))  
- Bug in date anchoring for `DateFrame.resample()` with offset `Day` when n > 1 ([GH24127](https://github.com/pandas-dev/pandas/issues/24127))  
- Bug where `ValueError` is wrongly raised when calling `count()` method of a
  `SeriesGroupBy` when the grouping variable only contains NaNs and numpy version < 1.13 ([GH21956](https://github.com/pandas-dev/pandas/issues/21956)).  
- Multiple bugs in `pandas.core.window.Rolling.min()` with `closed='left'` and a
  datetime-like index leading to incorrect results and also segfault. ([GH21704](https://github.com/pandas-dev/pandas/issues/21704))  
- Bug in `pandas.core.resample.Resampler.apply()` when passing positional arguments to applied func ([GH14615](https://github.com/pandas-dev/pandas/issues/14615)).  
- Bug in `Series.resample()` when passing `numpy.timedelta64` to `loffset` kwarg ([GH7687](https://github.com/pandas-dev/pandas/issues/7687)).  
- Bug in `pandas.core.resample.Resampler.asfreq()` when frequency of `TimedeltaIndex` is a subperiod of a new frequency ([GH13022](https://github.com/pandas-dev/pandas/issues/13022)).  
- Bug in `pandas.core.groupby.SeriesGroupBy.mean()` when values were integral but could not fit inside of int64, overflowing instead. ([GH22487](https://github.com/pandas-dev/pandas/issues/22487))  
- `pandas.core.groupby.RollingGroupby.agg()` and `pandas.core.groupby.ExpandingGroupby.agg()` now support multiple aggregation functions as parameters ([GH15072](https://github.com/pandas-dev/pandas/issues/15072))  
- Bug in `DataFrame.resample()` and `Series.resample()` when resampling by a weekly offset (`'W'`) across a DST transition ([GH9119](https://github.com/pandas-dev/pandas/issues/9119), [GH21459](https://github.com/pandas-dev/pandas/issues/21459))  
- Bug in `DataFrame.expanding()` in which the `axis` argument was not being respected during aggregations ([GH23372](https://github.com/pandas-dev/pandas/issues/23372))  
- Bug in `pandas.core.groupby.GroupBy.transform()` which caused missing values when the input function can accept a `DataFrame` but renames it ([GH23455](https://github.com/pandas-dev/pandas/issues/23455)).  
- Bug in `pandas.core.groupby.GroupBy.nth()` where column order was not always preserved ([GH20760](https://github.com/pandas-dev/pandas/issues/20760))  
- Bug in `pandas.core.groupby.GroupBy.rank()` with `method='dense'` and `pct=True` when a group has only one member would raise a `ZeroDivisionError` ([GH23666](https://github.com/pandas-dev/pandas/issues/23666)).  
- Calling `pandas.core.groupby.GroupBy.rank()` with empty groups and `pct=True` was raising a `ZeroDivisionError` ([GH22519](https://github.com/pandas-dev/pandas/issues/22519))  
- Bug in `DataFrame.resample()` when resampling `NaT` in `TimeDeltaIndex` ([GH13223](https://github.com/pandas-dev/pandas/issues/13223)).  
- Bug in `DataFrame.groupby()` did not respect the `observed` argument when selecting a column and instead always used `observed=False` ([GH23970](https://github.com/pandas-dev/pandas/issues/23970))  
- Bug in `pandas.core.groupby.SeriesGroupBy.pct_change()` or `pandas.core.groupby.DataFrameGroupBy.pct_change()` would previously work across groups when calculating the percent change, where it now correctly works per group ([GH21200](https://github.com/pandas-dev/pandas/issues/21200), [GH21235](https://github.com/pandas-dev/pandas/issues/21235)).  
- Bug preventing hash table creation with very large number (2^32) of rows ([GH22805](https://github.com/pandas-dev/pandas/issues/22805))  
- Bug in groupby when grouping on categorical causes `ValueError` and incorrect grouping if `observed=True` and `nan` is present in categorical column ([GH24740](https://github.com/pandas-dev/pandas/issues/24740), [GH21151](https://github.com/pandas-dev/pandas/issues/21151)).  

### Reshaping

- Bug in `pandas.concat()` when joining resampled DataFrames with timezone aware index ([GH13783](https://github.com/pandas-dev/pandas/issues/13783))  
- Bug in `pandas.concat()` when joining only Series the names argument of concat is no longer ignored ([GH23490](https://github.com/pandas-dev/pandas/issues/23490))  
- Bug in `Series.combine_first()` with `datetime64[ns, tz]` dtype which would return tz-naive result ([GH21469](https://github.com/pandas-dev/pandas/issues/21469))  
- Bug in `Series.where()` and `DataFrame.where()` with `datetime64[ns, tz]` dtype ([GH21546](https://github.com/pandas-dev/pandas/issues/21546))  
- Bug in `DataFrame.where()` with an empty DataFrame and empty `cond` having non-bool dtype ([GH21947](https://github.com/pandas-dev/pandas/issues/21947))  
- Bug in `Series.mask()` and `DataFrame.mask()` with `list` conditionals ([GH21891](https://github.com/pandas-dev/pandas/issues/21891))  
- Bug in `DataFrame.replace()` raises RecursionError when converting OutOfBounds `datetime64[ns, tz]` ([GH20380](https://github.com/pandas-dev/pandas/issues/20380))  
- `pandas.core.groupby.GroupBy.rank()` now raises a `ValueError` when an invalid value is passed for argument `na_option` ([GH22124](https://github.com/pandas-dev/pandas/issues/22124))  
- Bug in `get_dummies()` with Unicode attributes in Python 2 ([GH22084](https://github.com/pandas-dev/pandas/issues/22084))  
- Bug in `DataFrame.replace()` raises `RecursionError` when replacing empty lists ([GH22083](https://github.com/pandas-dev/pandas/issues/22083))  
- Bug in `Series.replace()` and `DataFrame.replace()` when dict is used as the `to_replace` value and one key in the dict is is another key’s value, the results were inconsistent between using integer key and using string key ([GH20656](https://github.com/pandas-dev/pandas/issues/20656))  
- Bug in `DataFrame.drop_duplicates()` for empty `DataFrame` which incorrectly raises an error ([GH20516](https://github.com/pandas-dev/pandas/issues/20516))  
- Bug in `pandas.wide_to_long()` when a string is passed to the stubnames argument and a column name is a substring of that stubname ([GH22468](https://github.com/pandas-dev/pandas/issues/22468))  
- Bug in `merge()` when merging `datetime64[ns, tz]` data that contained a DST transition ([GH18885](https://github.com/pandas-dev/pandas/issues/18885))  
- Bug in `merge_asof()` when merging on float values within defined tolerance ([GH22981](https://github.com/pandas-dev/pandas/issues/22981))  
- Bug in `pandas.concat()` when concatenating a multicolumn DataFrame with tz-aware data against a DataFrame with a different number of columns ([GH22796](https://github.com/pandas-dev/pandas/issues/22796))  
- Bug in `merge_asof()` where confusing error message raised when attempting to merge with missing values ([GH23189](https://github.com/pandas-dev/pandas/issues/23189))  
- Bug in `DataFrame.nsmallest()` and `DataFrame.nlargest()` for dataframes that have a `MultiIndex` for columns ([GH23033](https://github.com/pandas-dev/pandas/issues/23033)).  
- Bug in `pandas.melt()` when passing column names that are not present in `DataFrame` ([GH23575](https://github.com/pandas-dev/pandas/issues/23575))  
- Bug in `DataFrame.append()` with a `Series` with a dateutil timezone would raise a `TypeError` ([GH23682](https://github.com/pandas-dev/pandas/issues/23682))  
- Bug in `Series` construction when passing no data and `dtype=str` ([GH22477](https://github.com/pandas-dev/pandas/issues/22477))  
- Bug in `cut()` with `bins` as an overlapping `IntervalIndex` where multiple bins were returned per item instead of raising a `ValueError` ([GH23980](https://github.com/pandas-dev/pandas/issues/23980))  
- Bug in `pandas.concat()` when joining `Series` datetimetz with `Series` category would lose timezone ([GH23816](https://github.com/pandas-dev/pandas/issues/23816))  
- Bug in `DataFrame.join()` when joining on partial MultiIndex would drop names ([GH20452](https://github.com/pandas-dev/pandas/issues/20452)).  
- `DataFrame.nlargest()` and `DataFrame.nsmallest()` now returns the correct n values when keep != ‘all’ also when tied on the first columns ([GH22752](https://github.com/pandas-dev/pandas/issues/22752))  
- Constructing a DataFrame with an index argument that wasn’t already an instance of `Index` was broken ([GH22227](https://github.com/pandas-dev/pandas/issues/22227)).  
- Bug in `DataFrame` prevented list subclasses to be used to construction ([GH21226](https://github.com/pandas-dev/pandas/issues/21226))  
- Bug in `DataFrame.unstack()` and `DataFrame.pivot_table()` returning a missleading error message when the resulting DataFrame has more elements than int32 can handle. Now, the error message is improved, pointing towards the actual problem ([GH20601](https://github.com/pandas-dev/pandas/issues/20601))  
- Bug in `DataFrame.unstack()` where a `ValueError` was raised when unstacking timezone aware values ([GH18338](https://github.com/pandas-dev/pandas/issues/18338))  
- Bug in `DataFrame.stack()` where timezone aware values were converted to timezone naive values ([GH19420](https://github.com/pandas-dev/pandas/issues/19420))  
- Bug in `merge_asof()` where a `TypeError` was raised when `by_col` were timezone aware values ([GH21184](https://github.com/pandas-dev/pandas/issues/21184))  
- Bug showing an incorrect shape when throwing error during `DataFrame` construction. ([GH20742](https://github.com/pandas-dev/pandas/issues/20742))  



<a id='whatsnew-0240-bug-fixes-sparse'></a>

### Sparse

- Updating a boolean, datetime, or timedelta column to be Sparse now works ([GH22367](https://github.com/pandas-dev/pandas/issues/22367))  
- Bug in `Series.to_sparse()` with Series already holding sparse data not constructing properly ([GH22389](https://github.com/pandas-dev/pandas/issues/22389))  
- Providing a `sparse_index` to the SparseArray constructor no longer defaults the na-value to `np.nan` for all dtypes. The correct na_value for `data.dtype` is now used.  
- Bug in `SparseArray.nbytes` under-reporting its memory usage by not including the size of its sparse index.  
- Improved performance of `Series.shift()` for non-NA `fill_value`, as values are no longer converted to a dense array.  
- Bug in `DataFrame.groupby` not including `fill_value` in the groups for non-NA `fill_value` when grouping by a sparse column ([GH5078](https://github.com/pandas-dev/pandas/issues/5078))  
- Bug in unary inversion operator (`~`) on a `SparseSeries` with boolean values. The performance of this has also been improved ([GH22835](https://github.com/pandas-dev/pandas/issues/22835))  
- Bug in `SparseArary.unique()` not returning the unique values ([GH19595](https://github.com/pandas-dev/pandas/issues/19595))  
- Bug in `SparseArray.nonzero()` and `SparseDataFrame.dropna()` returning shifted/incorrect results ([GH21172](https://github.com/pandas-dev/pandas/issues/21172))  
- Bug in `DataFrame.apply()` where dtypes would lose sparseness ([GH23744](https://github.com/pandas-dev/pandas/issues/23744))  
- Bug in `concat()` when concatenating a list of `Series` with all-sparse values changing the `fill_value` and converting to a dense Series ([GH24371](https://github.com/pandas-dev/pandas/issues/24371))  

### Style

- `background_gradient()` now takes a `text_color_threshold` parameter to automatically lighten the text color based on the luminance of the background color. This improves readability with dark background colors without the need to limit the background colormap range. ([GH21258](https://github.com/pandas-dev/pandas/issues/21258))  
- `background_gradient()` now also supports tablewise application (in addition to rowwise and columnwise) with `axis=None` ([GH15204](https://github.com/pandas-dev/pandas/issues/15204))  
- `bar()` now also supports tablewise application (in addition to rowwise and columnwise) with `axis=None` and setting clipping range with `vmin` and `vmax` ([GH21548](https://github.com/pandas-dev/pandas/issues/21548) and [GH21526](https://github.com/pandas-dev/pandas/issues/21526)). `NaN` values are also handled properly.  

### Build changes

- Building pandas for development now requires `cython >= 0.28.2` ([GH21688](https://github.com/pandas-dev/pandas/issues/21688))  
- Testing pandas now requires `hypothesis>=3.58`.  You can find [the Hypothesis docs here](https://hypothesis.readthedocs.io/en/latest/index.html), and a pandas-specific introduction [in the contributing guide](development/contributing.ipynb#using-hypothesis). ([GH22280](https://github.com/pandas-dev/pandas/issues/22280))  
- Building pandas on macOS now targets minimum macOS 10.9 if run on macOS 10.9 or above ([GH23424](https://github.com/pandas-dev/pandas/issues/23424))  

### Other

- Bug where C variables were declared with external linkage causing import errors if certain other C libraries were imported before Pandas. ([GH24113](https://github.com/pandas-dev/pandas/issues/24113))  



<a id='whatsnew-0-24-0-contributors'></a>

## Contributors

A total of 516 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- 1_x7 +  
- AJ Dyka +  
- AJ Pryor, Ph.D +  
- Aaron Critchley  
- Abdullah İhsan Seçer +  
- Adam Bull +  
- Adam Hooper  
- Adam J. Stewart  
- Adam Kim  
- Adam Klimont +  
- Addison Lynch +  
- Alan Hogue +  
- Albert Villanova del Moral  
- Alex Radu +  
- Alex Rychyk  
- Alex Strick van Linschoten +  
- Alex Volkov +  
- Alex Watt +  
- AlexTereshenkov +  
- Alexander Buchkovsky  
- Alexander Hendorf +  
- Alexander Hess +  
- Alexander Nordin +  
- Alexander Ponomaroff +  
- Alexandre Batisse +  
- Alexandre Decan +  
- Allen Downey +  
- Allison Browne +  
- Aly Sivji  
- Alyssa Fu Ward +  
- Andrew  
- Andrew Gaspari +  
- Andrew Gross +  
- Andrew Spott +  
- Andrew Wood +  
- Andy +  
- Aniket uttam +  
- Anjali2019 +  
- Anjana S +  
- Antoine Viscardi +  
- Antonio Gutierrez +  
- Antti Kaihola +  
- Anudeep Tubati +  
- Arjun Sharma +  
- Armin Varshokar  
- Arno Veenstra +  
- Artem Bogachev  
- ArtinSarraf +  
- Barry Fitzgerald +  
- Bart Aelterman +  
- Batalex +  
- Baurzhan Muftakhidinov  
- Ben James +  
- Ben Nelson +  
- Benjamin Grove +  
- Benjamin Rowell +  
- Benoit Paquet +  
- Bharat Raghunathan +  
- Bhavani Ravi +  
- Big Head +  
- Boris Lau +  
- Brett Naul  
- Brett Randall +  
- Brian Choi +  
- Bryan Cutler +  
- C John Klehm +  
- C.A.M. Gerlach +  
- Caleb Braun +  
- Carl Johan +  
- Cecilia +  
- Chalmer Lowe  
- Chang She  
- Charles David +  
- Cheuk Ting Ho  
- Chris  
- Chris Bertinato +  
- Chris Roberts +  
- Chris Stadler +  
- Christian Haege +  
- Christian Hudon  
- Christopher Whelan  
- Chu Qing Hao +  
- Chuanzhu Xu +  
- Clemens Brunner  
- Da Cheezy Mobsta +  
- Damian Kula +  
- Damini Satya  
- Daniel Himmelstein  
- Daniel Hrisca +  
- Daniel Luis Costa +  
- Daniel Saxton +  
- DanielFEvans +  
- Darcy Meyer +  
- DataOmbudsman  
- David Arcos  
- David Krych  
- David Liu +  
- Dean Langsam +  
- Deepyaman Datta +  
- Denis Belavin +  
- Devin Petersohn +  
- Diane Trout +  
- Diego Argueta +  
- Diego Torres +  
- Dobatymo +  
- Doug Latornell +  
- Dr. Irv  
- Dylan Dmitri Gray +  
- EdAbati +  
- Enrico Rotundo +  
- Eric Boxer +  
- Eric Chea  
- Erik +  
- Erik Nilsson +  
- EternalLearner42 +  
- Evan +  
- Evan Livelo +  
- Fabian Haase +  
- Fabian Retkowski  
- Fabian Rost +  
- Fabien Aulaire +  
- Fakabbir Amin +  
- Fei Phoon +  
- Fernando Margueirat +  
- Flavien Lambert +  
- Florian Müller +  
- Florian Rathgeber +  
- Frank Hoang +  
- Fábio Rosado +  
- Gabe Fernando  
- Gabriel Reid +  
- Gaibo Zhang +  
- Giftlin Rajaiah  
- Gioia Ballin +  
- Giuseppe Romagnuolo +  
- Gjelt  
- Gordon Blackadder +  
- Gosuke Shibahara +  
- Graham Inggs  
- Gregory Rome +  
- Guillaume Gay  
- Guillaume Lemaitre +  
- HHest +  
- Hannah Ferchland  
- Haochen Wu  
- Hielke Walinga +  
- How Si Wei +  
- Hubert +  
- HubertKl +  
- Huize Wang +  
- Hyukjin Kwon +  
- HyunTruth +  
- Iain Barr  
- Ian Dunn +  
- Ignacio Vergara Kausel +  
- Inevitable-Marzipan +  
- Irv Lustig +  
- IsvenC +  
- JElfner +  
- Jacob Bundgaard +  
- Jacopo Rota  
- Jakob Jarmar +  
- James Bourbeau +  
- James Cobon-Kerr +  
- James Myatt +  
- James Winegar +  
- Jan Rudolph  
- Jan-Philip Gehrcke +  
- Jared Groves +  
- Jarrod Millman +  
- Jason Kiley +  
- Javad Noorbakhsh +  
- Jay Offerdahl +  
- Jayanth Katuri +  
- Jeff Reback  
- Jeongmin Yu +  
- Jeremy Schendel  
- Jerod Estapa +  
- Jesper Dramsch +  
- Jiang Yue +  
- Jim Jeon +  
- Joe Jevnik  
- Joel Nothman  
- Joel Ostblom +  
- Johan von Forstner +  
- Johnny Chiu +  
- Jonas +  
- Jonathon Vandezande +  
- Jop Vermeer +  
- Jordi Contestí  
- Jorge López Fueyo +  
- Joris Van den Bossche  
- Jose Quinones +  
- Jose Rivera-Rubio +  
- Josh  
- Josh Friedlander +  
- Jun +  
- Justin Zheng +  
- Kaiqi Dong +  
- Kalyan Gokhale  
- Kane +  
- Kang Yoosam +  
- Kapil Patel +  
- Kara de la Marck +  
- Karl Dunkle Werner +  
- Karmanya Aggarwal +  
- Katherine Surta +  
- Katrin Leinweber +  
- Kendall Masse +  
- Kevin Markham +  
- Kevin Sheppard  
- Kimi Li +  
- Koustav Samaddar +  
- Krishna +  
- Kristian Holsheimer +  
- Ksenia Gueletina +  
- Kyle Kosic +  
- Kyle Prestel +  
- LJ +  
- LeakedMemory +  
- Li Jin +  
- Licht Takeuchi  
- Lorenzo Stella +  
- Luca Donini +  
- Luciano Viola +  
- Maarten Rietbergen +  
- Mak Sze Chun +  
- Marc Garcia  
- Marius Potgieter +  
- Mark Sikora +  
- Markus Meier +  
- Marlene Silva Marchena +  
- Martin Babka +  
- MatanCohe +  
- Mateusz Woś +  
- Mathew Topper +  
- Matias Heikkilä  
- Mats Maiwald +  
- Matt Boggess +  
- Matt Cooper +  
- Matt Williams +  
- Matthew Gilbert  
- Matthew Roeschke  
- Max Bolingbroke +  
- Max Kanter  
- Max Kovalovs +  
- Max van Deursen +  
- Michael  
- Michael Davis +  
- Michael Odintsov  
- Michael P. Moran +  
- Michael Silverstein +  
- Michael-J-Ward +  
- Mickaël Schoentgen +  
- Miguel Sánchez de León Peque +  
- Mike Cramblett +  
- Min ho Kim +  
- Ming Li  
- Misha Veldhoen +  
- Mitar  
- Mitch Negus  
- Monson Shao +  
- Moonsoo Kim +  
- Mortada Mehyar  
- Mukul Ashwath Ram +  
- MusTheDataGuy +  
- Myles Braithwaite  
- Nanda H Krishna +  
- Nehil Jain +  
- Nicholas Musolino +  
- Nicolas Dickreuter +  
- Nikhil Kumar Mengani +  
- Nikoleta Glynatsi +  
- Noam Hershtig +  
- Noora Husseini +  
- Ondrej Kokes  
- Pablo Ambrosio +  
- Pamela Wu +  
- Parfait G +  
- Patrick Park +  
- Paul  
- Paul Ganssle  
- Paul Reidy  
- Paul van Mulbregt +  
- Pauli Virtanen  
- Pav A +  
- Philippe Ombredanne +  
- Phillip Cloud  
- Pietro Battiston  
- Piyush Aggarwal +  
- Prabakaran Kumaresshan +  
- Pulkit Maloo  
- Pyry Kovanen  
- Rajib Mitra +  
- Redonnet Louis +  
- Rhys Parry +  
- Richard Eames +  
- Rick +  
- Robin  
- Roei.r +  
- RomainSa +  
- Roman Imankulov +  
- Roman Yurchak +  
- Ruijing Li +  
- Ryan +  
- Ryan Joyce +  
- Ryan Nazareth +  
- Ryan Rehman +  
- Rüdiger Busche +  
- SEUNG HOON, SHIN +  
- Sakar Panta +  
- Samuel Sinayoko  
- Sandeep Pathak +  
- Sandrine Pataut +  
- Sangwoong Yoon  
- Santosh Kumar +  
- Saurav Chakravorty +  
- Scott McAllister +  
- Scott Talbert +  
- Sean Chan +  
- Sergey Kopylov +  
- Shadi Akiki +  
- Shantanu Gontia +  
- Shengpu Tang +  
- Shirish Kadam +  
- Shivam Rana +  
- Shorokhov Sergey +  
- Simon Hawkins +  
- Simon Riddell +  
- Simone Basso  
- Sinhrks  
- Soyoun(Rose) Kim +  
- Srinivas Reddy Thatiparthy (శ్రీనివాస్  రెడ్డి తాటిపర్తి) +  
- Stefaan Lippens +  
- Stefano Cianciulli  
- Stefano Miccoli +  
- Stephan Hoyer  
- Stephen Childs  
- Stephen Cowley +  
- Stephen Pascoe  
- Stephen Rauch  
- Sterling Paramore +  
- Steve Baker +  
- Steve Cook +  
- Steve Dower +  
- Steven +  
- Stijn Van Hoey  
- Stéphan Taljaard +  
- Sumanau Sareen +  
- Sumin Byeon +  
- Sören +  
- Takuya N +  
- Tamas Nagy +  
- Tan Tran +  
- Tanya Jain +  
- Tao He +  
- Tarbo Fukazawa  
- Terji Petersen +  
- Thein Oo +  
- Thiago Cordeiro da Fonseca +  
- ThibTrip +  
- Thierry Moisan  
- Thijs Damsma +  
- Thiviyan Thanapalasingam +  
- Thomas A Caswell  
- Thomas Kluiters +  
- Thomas Lentali +  
- Tilen Kusterle +  
- Tim D. Smith +  
- Tim Gates +  
- Tim Hoffmann  
- Tim Swast  
- Tom Augspurger  
- Tom Neep +  
- Tomasz Kluczkowski +  
- Tomáš Chvátal +  
- Tony Tao +  
- Triple0 +  
- Troels Nielsen +  
- Tuhin Mahmud +  
- Tyler Reddy +  
- Uddeshya Singh  
- Uwe L. Korn +  
- Vadym Barda +  
- Vaibhav Vishal +  
- Varad Gunjal +  
- Vasily Litvinov +  
- Vibhu Agarwal +  
- Victor Maryama +  
- Victor Villas  
- Vikramjeet Das +  
- Vincent La  
- Vitória Helena +  
- Vladislav +  
- Vu Le  
- Vyom Jain +  
- Víctor Moron Tejero +  
- Weiwen Gu +  
- Wenhuan  
- Wes Turner  
- Wil Tan +  
- Will Ayd +  
- William Ayd  
- Wouter De Coster +  
- Yeojin Kim +  
- Yitzhak Andrade +  
- Yoann Goular +  
- Yuecheng Wu +  
- Yuliya Dovzhenko +  
- Yury Bayda +  
- Zac Hatfield-Dodds +  
- Zach Angell +  
- aberres +  
- aeltanawy +  
- ailchau +  
- alimcmaster1  
- alphaCTzo7G +  
- amphy +  
- anmyachev +  
- araraonline +  
- azure-pipelines[bot] +  
- benarthur91 +  
- bk521234 +  
- cgangwar11 +  
- chris-b1  
- cxl923cc +  
- dahlbaek +  
- danielplawrence +  
- dannyhyunkim +  
- darke-spirits +  
- david-liu-brattle-1  
- davidmvalente +  
- deflatSOCO  
- doosik_bae +  
- dylanchase +  
- eduardo naufel schettino +  
- endenis +  
- enisnazif +  
- euri10 +  
- evangelineliu +  
- ezcitron +  
- fengyqf +  
- fjdiod  
- fjetter  
- fl4p +  
- fleimgruber +  
- froessler  
- gfyoung  
- gwrome +  
- h-vetinari  
- haison +  
- hannah-c +  
- harisbal +  
- heckeop +  
- henriqueribeiro +  
- himanshu awasthi  
- hongshaoyang +  
- iamshwin +  
- igorfassen +  
- jalazbe +  
- jamesoliverh +  
- jbrockmendel  
- jh-wu +  
- jkovacevic +  
- justinchan23 +  
- killerontherun1 +  
- knuu +  
- kpapdac +  
- kpflugshaupt +  
- krsnik93 +  
- leerssej +  
- louispotok  
- lrjball +  
- marcosrullan +  
- mazayo +  
- miker985  
- nathalier +  
- nicolab100 +  
- nprad  
- nrebena +  
- nsuresh +  
- nullptr +  
- ottiP  
- pajachiet +  
- pilkibun +  
- pmaxey83 +  
- raguiar2 +  
- ratijas +  
- rbenes +  
- realead +  
- robbuckley +  
- saurav2608 +  
- shawnbrown +  
- sideeye +  
- ssikdar1  
- sudhir mohanraj +  
- svenharris +  
- syutbai +  
- tadeja +  
- tamuhey +  
- testvinder +  
- thatneat  
- tmnhat2001  
- tomascassidy +  
- tomneep  
- topper-123  
- vkk800 +  
- willweil +  
- winlu +  
- yehia67 +  
- yhaque1213 +  
- ym-pett +  
- yrhooke +  
- ywpark1 +  
- zertrin  
- zhezherun +  